<a href="https://colab.research.google.com/github/Alex-Jung-HB/0816_python_Pre-trained-Cityscapes-Model/blob/main/0816_python_Pre_trained_Cityscapes_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-trained Cityscapes Model:

In [4]:
# ===============================================================================
# SEGFORMER QUALITY CHECK SYSTEM - STEP BY STEP VALIDATION
# ===============================================================================
# This enhanced version provides comprehensive quality checking before video processing
# ===============================================================================

import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import cv2
import matplotlib.pyplot as plt
import os
from datetime import datetime
import json
import time

# For Colab environment
try:
    from google.colab import files
    from IPython.display import display, Image as IPImage, HTML
    COLAB_ENV = True
    print("✅ Google Colab environment detected")
except ImportError:
    COLAB_ENV = False
    print("📱 Running in local environment")

# ===============================================================================
# STEP 1: QUALITY CHECKER CLASS
# ===============================================================================
class SegmentationQualityChecker:
    """Comprehensive quality checking system for segmentation results"""

    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.processor = None
        self.model = None

        # Cityscapes classes for evaluation
        self.cityscapes_classes = {
            0: 'road', 1: 'sidewalk', 2: 'building', 3: 'wall', 4: 'fence',
            5: 'pole', 6: 'traffic_light', 7: 'traffic_sign', 8: 'vegetation',
            9: 'terrain', 10: 'sky', 11: 'person', 12: 'rider', 13: 'car',
            14: 'truck', 15: 'bus', 16: 'train', 17: 'motorcycle', 18: 'bicycle'
        }

        # Color mapping for clear visualization
        self.color_map = {
            0: [128, 64, 128],   # road - purple
            1: [244, 35, 232],   # sidewalk - pink
            2: [70, 70, 70],     # building - gray
            5: [153, 153, 153],  # pole - light gray
            6: [250, 170, 30],   # traffic_light - orange
            7: [220, 220, 0],    # traffic_sign - yellow
            8: [107, 142, 35],   # vegetation - olive
            10: [70, 130, 180],  # sky - steel blue
            11: [220, 20, 60],   # person - crimson
            13: [0, 0, 142],     # car - dark blue
            14: [0, 0, 70],      # truck - darker blue
            15: [0, 60, 100],    # bus - navy
            17: [0, 0, 230],     # motorcycle - blue
            18: [119, 11, 32],   # bicycle - dark red
        }

        # Quality criteria for traffic scenes
        self.quality_criteria = {
            'road_coverage': {'min': 15, 'max': 70, 'weight': 0.3},          # 15-70% road coverage
            'fragmentation': {'max_fragments': 50, 'weight': 0.25},          # Low fragmentation
            'boundary_quality': {'min_coherence': 0.7, 'weight': 0.2},      # Smooth boundaries
            'traffic_elements': {'min_detection': 1, 'weight': 0.15},       # Detect traffic elements
            'background_ratio': {'min': 20, 'max': 80, 'weight': 0.1}       # Reasonable background
        }

        self.setup_directories()

    def setup_directories(self):
        """Create output directories"""
        self.output_dir = "./segmentation_quality_check"
        self.sample_dir = os.path.join(self.output_dir, "samples")
        self.results_dir = os.path.join(self.output_dir, "results")

        for dir_path in [self.output_dir, self.sample_dir, self.results_dir]:
            os.makedirs(dir_path, exist_ok=True)

    def load_model(self):
        """Load pre-trained Segformer model"""
        print("🔄 Loading Segformer model...")
        print("=" * 60)

        try:
            model_name = "nvidia/segformer-b0-finetuned-cityscapes-1024-1024"
            self.processor = SegformerImageProcessor.from_pretrained(model_name)
            self.model = SegformerForSemanticSegmentation.from_pretrained(model_name)
            self.model.to(self.device)
            self.model.eval()

            print("✅ Model loaded successfully!")
            print(f"📱 Device: {self.device}")
            print(f"🎯 Classes: {len(self.cityscapes_classes)}")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            return False

# ===============================================================================
# STEP 2: SAMPLE IMAGE GENERATOR
# ===============================================================================
    def create_traffic_sample_images(self):
        """Create realistic traffic scene sample images for testing"""
        print("\n🎨 CREATING TRAFFIC SAMPLE IMAGES")
        print("=" * 60)

        samples = []

        # Sample 1: Highway scene
        highway_img = self._create_highway_scene()
        highway_path = os.path.join(self.sample_dir, "sample_highway.jpg")
        highway_img.save(highway_path)
        samples.append(("Highway Scene", highway_path))

        # Sample 2: City intersection
        city_img = self._create_city_intersection()
        city_path = os.path.join(self.sample_dir, "sample_city.jpg")
        city_img.save(city_path)
        samples.append(("City Intersection", city_path))

        # Sample 3: Traffic with signs
        traffic_img = self._create_traffic_signs_scene()
        traffic_path = os.path.join(self.sample_dir, "sample_traffic_signs.jpg")
        traffic_img.save(traffic_path)
        samples.append(("Traffic Signs Scene", traffic_path))

        print(f"✅ Created {len(samples)} sample images:")
        for name, path in samples:
            print(f"  • {name}: {path}")

        return samples

    def _create_highway_scene(self):
        """Create a highway-like sample image"""
        img = Image.new('RGB', (800, 600), color=(135, 206, 235))  # Sky blue background
        draw = ImageDraw.Draw(img)

        # Road surface (gray)
        draw.rectangle([0, 400, 800, 600], fill=(70, 70, 70))

        # Lane markings (white)
        for x in range(50, 800, 100):
            draw.rectangle([x, 480, x+40, 490], fill=(255, 255, 255))

        # Side barriers (concrete)
        draw.rectangle([0, 380, 800, 400], fill=(150, 150, 150))

        # Vehicles (simplified rectangles)
        # Car 1
        draw.rectangle([200, 420, 280, 470], fill=(255, 0, 0))  # Red car
        # Car 2
        draw.rectangle([450, 430, 530, 480], fill=(0, 0, 255))  # Blue car

        # Trees/vegetation on sides
        for x in range(100, 800, 150):
            draw.ellipse([x, 200, x+60, 280], fill=(34, 139, 34))  # Green circles

        return img

    def _create_city_intersection(self):
        """Create a city intersection sample"""
        img = Image.new('RGB', (800, 600), color=(135, 206, 235))  # Sky
        draw = ImageDraw.Draw(img)

        # Buildings (gray/brown)
        draw.rectangle([0, 0, 300, 350], fill=(139, 69, 19))      # Left building
        draw.rectangle([500, 0, 800, 300], fill=(105, 105, 105))  # Right building

        # Road intersection
        draw.rectangle([300, 350, 500, 600], fill=(70, 70, 70))   # Vertical road
        draw.rectangle([0, 450, 800, 550], fill=(70, 70, 70))     # Horizontal road

        # Crosswalk (white stripes)
        for y in range(460, 540, 8):
            draw.rectangle([320, y, 480, y+4], fill=(255, 255, 255))

        # Traffic light (simplified)
        draw.rectangle([290, 330, 310, 380], fill=(128, 128, 128))  # Pole
        draw.ellipse([285, 320, 315, 350], fill=(255, 255, 0))      # Yellow light

        # Stop sign
        draw.polygon([(480, 300), (520, 320), (520, 360), (480, 380),
                     (440, 360), (440, 320)], fill=(255, 0, 0))

        return img

    def _create_traffic_signs_scene(self):
        """Create scene with various traffic elements"""
        img = Image.new('RGB', (800, 600), color=(135, 206, 235))  # Sky
        draw = ImageDraw.Draw(img)

        # Road
        draw.rectangle([0, 400, 800, 600], fill=(70, 70, 70))

        # Sidewalk
        draw.rectangle([0, 350, 800, 400], fill=(244, 164, 96))

        # Various traffic signs
        # Speed limit sign (rectangular)
        draw.rectangle([100, 200, 180, 280], fill=(255, 255, 255))
        draw.rectangle([105, 205, 175, 275], fill=(255, 0, 0))

        # Warning sign (triangular)
        draw.polygon([(300, 180), (250, 280), (350, 280)], fill=(255, 255, 0))

        # Traffic light pole
        draw.rectangle([450, 100, 470, 350], fill=(128, 128, 128))
        draw.rectangle([430, 100, 490, 160], fill=(0, 0, 0))
        draw.ellipse([435, 105, 455, 125], fill=(255, 0, 0))      # Red
        draw.ellipse([435, 120, 455, 140], fill=(255, 255, 0))    # Yellow
        draw.ellipse([435, 135, 455, 155], fill=(0, 255, 0))      # Green

        # Vehicles
        draw.rectangle([600, 420, 720, 480], fill=(0, 0, 139))    # Car

        return img

# ===============================================================================
# STEP 3: QUALITY ASSESSMENT FUNCTIONS
# ===============================================================================
    def predict_and_analyze(self, image_path_or_pil):
        """Predict segmentation and perform detailed analysis"""
        try:
            # Load image
            if isinstance(image_path_or_pil, str):
                image = Image.open(image_path_or_pil).convert("RGB")
            else:
                image = image_path_or_pil.convert("RGB")

            # Predict
            inputs = self.processor(image, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)
                predictions = torch.nn.functional.interpolate(
                    outputs.logits,
                    size=image.size[::-1],
                    mode="bilinear",
                    align_corners=False,
                )
                predicted_map = predictions.squeeze().cpu().numpy().argmax(axis=0)

            return self._analyze_prediction_quality(predicted_map, image)

        except Exception as e:
            print(f"❌ Error during prediction: {str(e)}")
            return None

    def _analyze_prediction_quality(self, prediction, image):
        """Comprehensive quality analysis of prediction"""
        h, w = prediction.shape
        total_pixels = h * w

        # Get class distribution
        unique_classes, counts = np.unique(prediction, return_counts=True)
        class_distribution = {}

        for class_id, count in zip(unique_classes, counts):
            percentage = (count / total_pixels) * 100
            class_name = self.cityscapes_classes.get(class_id, f"Unknown_{class_id}")
            class_distribution[class_name] = {
                'pixels': int(count),
                'percentage': round(percentage, 2),
                'class_id': int(class_id)
            }

        # Quality metrics
        quality_scores = self._calculate_quality_scores(prediction, class_distribution)

        # Create visualizations
        colored_mask = self._create_colored_mask(prediction)
        blended = self._create_blended_image(image, colored_mask)

        return {
            'prediction': prediction,
            'image': image,
            'colored_mask': colored_mask,
            'blended': blended,
            'class_distribution': class_distribution,
            'quality_scores': quality_scores,
            'dimensions': (h, w),
            'total_pixels': total_pixels
        }

    def _calculate_quality_scores(self, prediction, class_distribution):
        """Calculate quality scores based on criteria"""
        scores = {}

        # 1. Road coverage analysis
        road_percentage = class_distribution.get('road', {}).get('percentage', 0)
        road_score = self._score_in_range(road_percentage,
                                        self.quality_criteria['road_coverage']['min'],
                                        self.quality_criteria['road_coverage']['max'])
        scores['road_coverage'] = {
            'score': road_score,
            'value': road_percentage,
            'status': 'Good' if road_score > 0.7 else 'Poor'
        }

        # 2. Fragmentation analysis
        fragmentation_score = self._analyze_fragmentation(prediction)
        scores['fragmentation'] = {
            'score': fragmentation_score,
            'status': 'Good' if fragmentation_score > 0.7 else 'Poor'
        }

        # 3. Traffic elements detection
        traffic_elements = ['traffic_light', 'traffic_sign', 'car', 'truck', 'bus']
        detected_traffic = sum(1 for elem in traffic_elements if elem in class_distribution)
        traffic_score = min(1.0, detected_traffic / 2)  # At least 2 traffic elements
        scores['traffic_detection'] = {
            'score': traffic_score,
            'detected': detected_traffic,
            'status': 'Good' if traffic_score > 0.5 else 'Poor'
        }

        # 4. Overall coherence
        coherence_score = self._analyze_spatial_coherence(prediction)
        scores['spatial_coherence'] = {
            'score': coherence_score,
            'status': 'Good' if coherence_score > 0.6 else 'Poor'
        }

        # Calculate overall quality score
        weights = [0.3, 0.25, 0.2, 0.25]  # Road, fragmentation, traffic, coherence
        overall_score = sum(s['score'] * w for s, w in zip(scores.values(), weights))

        scores['overall'] = {
            'score': overall_score,
            'grade': self._get_quality_grade(overall_score),
            'status': 'Good' if overall_score > 0.6 else 'Poor'
        }

        return scores

    def _score_in_range(self, value, min_val, max_val):
        """Score value based on optimal range"""
        if min_val <= value <= max_val:
            return 1.0
        elif value < min_val:
            return max(0, value / min_val)
        else:
            return max(0, 1 - (value - max_val) / max_val)

    def _analyze_fragmentation(self, prediction):
        """Analyze image fragmentation (lower is better)"""
        # Count connected components for major classes
        fragmentation_penalty = 0

        for class_id in [0, 6, 7, 13]:  # road, traffic_light, traffic_sign, car
            mask = (prediction == class_id).astype(np.uint8)
            if np.sum(mask) > 100:  # Only analyze if significant presence
                num_labels, _ = cv2.connectedComponents(mask)
                if num_labels > 10:  # Too many fragments
                    fragmentation_penalty += (num_labels - 10) / 50

        return max(0, 1 - fragmentation_penalty)

    def _analyze_spatial_coherence(self, prediction):
        """Analyze spatial coherence of segmentation"""
        # Check for spatial coherence using edge consistency
        edges = cv2.Canny(prediction.astype(np.uint8), 1, 3)
        edge_ratio = np.sum(edges > 0) / prediction.size

        # Good segmentation should have moderate edge ratio (not too fragmented)
        if 0.05 <= edge_ratio <= 0.2:
            return 1.0
        elif edge_ratio < 0.05:
            return edge_ratio / 0.05
        else:
            return max(0, 1 - (edge_ratio - 0.2) / 0.3)

    def _get_quality_grade(self, score):
        """Convert score to letter grade"""
        if score >= 0.9: return 'A+'
        elif score >= 0.8: return 'A'
        elif score >= 0.7: return 'B+'
        elif score >= 0.6: return 'B'
        elif score >= 0.5: return 'C'
        else: return 'D'

    def _create_colored_mask(self, prediction):
        """Create colored visualization mask"""
        h, w = prediction.shape
        colored_mask = np.zeros((h, w, 3), dtype=np.uint8)

        for class_id, color in self.color_map.items():
            mask = prediction == class_id
            colored_mask[mask] = color

        return colored_mask

    def _create_blended_image(self, image, colored_mask, alpha=0.6):
        """Create blended image with segmentation overlay"""
        image_np = np.array(image)
        return cv2.addWeighted(image_np, alpha, colored_mask, 1-alpha, 0)

# ===============================================================================
# STEP 4: COMPREHENSIVE TESTING INTERFACE
# ===============================================================================
    def run_step_by_step_validation(self):
        """Run complete step-by-step validation process"""
        print("\n🎯 STEP-BY-STEP SEGMENTATION QUALITY VALIDATION")
        print("=" * 70)

        # Step 1: Model loading
        print("\n📥 STEP 1: Loading Model")
        print("-" * 40)
        if not self.load_model():
            return False

        # Step 2: Create sample images
        print("\n🎨 STEP 2: Creating Sample Images")
        print("-" * 40)
        samples = self.create_traffic_sample_images()

        # Step 3: Test each sample
        print("\n🔍 STEP 3: Testing Sample Images")
        print("-" * 40)

        all_results = []

        for i, (name, sample_path) in enumerate(samples, 1):
            print(f"\n🧪 Testing Sample {i}: {name}")
            print("." * 30)

            result = self.predict_and_analyze(sample_path)
            if result:
                # Save results
                self._save_detailed_results(result, f"sample_{i}_{name.lower().replace(' ', '_')}")
                all_results.append((name, result))

                # Print quick summary
                quality = result['quality_scores']['overall']
                print(f"   Quality Score: {quality['score']:.2f} (Grade: {quality['grade']})")
                print(f"   Status: {'✅ GOOD' if quality['status'] == 'Good' else '⚠️ NEEDS IMPROVEMENT'}")

        # Step 4: Overall assessment
        print("\n📊 STEP 4: Overall Assessment")
        print("-" * 40)

        average_score = np.mean([r[1]['quality_scores']['overall']['score'] for _, r in all_results])
        overall_grade = self._get_quality_grade(average_score)

        print(f"📈 Average Quality Score: {average_score:.2f}")
        print(f"🎓 Overall Grade: {overall_grade}")

        # Decision recommendation
        if average_score >= 0.6:
            recommendation = "✅ READY FOR VIDEO PROCESSING"
            print(f"\n🚀 {recommendation}")
            print("The model shows good segmentation quality. You can proceed with video processing.")
        else:
            recommendation = "⚠️ CONSIDER IMPROVEMENTS BEFORE VIDEO PROCESSING"
            print(f"\n⚠️ {recommendation}")
            print("The model quality may need improvement. Consider:")
            print("  • Using a different pre-trained model")
            print("  • Fine-tuning on your specific traffic data")
            print("  • Adjusting image preprocessing")

        # Generate summary report
        self._generate_summary_report(all_results, average_score, overall_grade, recommendation)

        return average_score >= 0.6

    def _save_detailed_results(self, result, prefix):
        """Save detailed results for each test"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Save images
        image_path = os.path.join(self.results_dir, f"{prefix}_original_{timestamp}.jpg")
        blended_path = os.path.join(self.results_dir, f"{prefix}_segmented_{timestamp}.jpg")
        mask_path = os.path.join(self.results_dir, f"{prefix}_mask_{timestamp}.jpg")

        # Convert and save
        result['image'].save(image_path)
        cv2.imwrite(blended_path, cv2.cvtColor(result['blended'], cv2.COLOR_RGB2BGR))
        cv2.imwrite(mask_path, cv2.cvtColor(result['colored_mask'], cv2.COLOR_RGB2BGR))

        # Save analysis JSON
        analysis_path = os.path.join(self.results_dir, f"{prefix}_analysis_{timestamp}.json")
        analysis_data = {
            'quality_scores': result['quality_scores'],
            'class_distribution': result['class_distribution'],
            'dimensions': result['dimensions'],
            'total_pixels': result['total_pixels'],
            'timestamp': timestamp
        }

        with open(analysis_path, 'w') as f:
            json.dump(analysis_data, f, indent=2)

        return {
            'image': image_path,
            'segmented': blended_path,
            'mask': mask_path,
            'analysis': analysis_path
        }

    def _generate_summary_report(self, all_results, average_score, overall_grade, recommendation):
        """Generate comprehensive summary report"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_path = os.path.join(self.results_dir, f"quality_validation_report_{timestamp}.html")

        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Segmentation Quality Validation Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                .header {{ background-color: #f0f0f0; padding: 20px; border-radius: 5px; }}
                .result {{ margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; }}
                .good {{ background-color: #e8f5e8; }}
                .poor {{ background-color: #ffe8e8; }}
                .score {{ font-size: 1.2em; font-weight: bold; }}
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>🎯 Segmentation Quality Validation Report</h1>
                <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
                <p><strong>Overall Score:</strong> <span class="score">{average_score:.2f} (Grade: {overall_grade})</span></p>
                <p><strong>Recommendation:</strong> {recommendation}</p>
            </div>
        """

        for name, result in all_results:
            quality = result['quality_scores']['overall']
            status_class = 'good' if quality['status'] == 'Good' else 'poor'

            html_content += f"""
            <div class="result {status_class}">
                <h2>📊 {name}</h2>
                <p><strong>Quality Score:</strong> {quality['score']:.2f} (Grade: {quality['grade']})</p>

                <h3>Detailed Scores:</h3>
                <table>
                    <tr><th>Metric</th><th>Score</th><th>Status</th><th>Details</th></tr>
            """

            for metric, data in result['quality_scores'].items():
                if metric != 'overall':
                    details = str(data.get('value', data.get('detected', '')))
                    html_content += f"""
                    <tr>
                        <td>{metric.replace('_', ' ').title()}</td>
                        <td>{data['score']:.2f}</td>
                        <td>{data['status']}</td>
                        <td>{details}</td>
                    </tr>
                    """

            html_content += """
                </table>

                <h3>Class Distribution:</h3>
                <table>
                    <tr><th>Class</th><th>Percentage</th><th>Pixels</th></tr>
            """

            for class_name, stats in result['class_distribution'].items():
                html_content += f"""
                <tr>
                    <td>{class_name}</td>
                    <td>{stats['percentage']}%</td>
                    <td>{stats['pixels']:,}</td>
                </tr>
                """

            html_content += "</table></div>"

        html_content += """
        </body>
        </html>
        """

        with open(report_path, 'w') as f:
            f.write(html_content)

        print(f"📄 Detailed report saved: {report_path}")

# ===============================================================================
# STEP 5: USER TESTING INTERFACE
# ===============================================================================
def test_user_image():
    """Test user-uploaded image with quality checking"""
    print("\n📤 USER IMAGE QUALITY TEST")
    print("=" * 60)

    checker = SegmentationQualityChecker()

    # Load model
    if not checker.load_model():
        return

    if COLAB_ENV:
        print("📁 Please upload your traffic image:")
        uploaded = files.upload()

        if not uploaded:
            print("❌ No file uploaded!")
            return

        image_path = list(uploaded.keys())[0]
    else:
        image_path = input("📁 Enter path to your traffic image: ").strip()
        if not os.path.exists(image_path):
            print(f"❌ File not found: {image_path}")
            return

    print(f"🔍 Analyzing: {image_path}")

    # Analyze image
    result = checker.predict_and_analyze(image_path)

    if result:
        # Save results
        saved_files = checker._save_detailed_results(result, "user_image")

        # Display analysis
        print("\n📊 QUALITY ANALYSIS RESULTS")
        print("=" * 60)

        quality = result['quality_scores']['overall']
        print(f"🎯 Overall Quality Score: {quality['score']:.2f}")
        print(f"🎓 Grade: {quality['grade']}")
        print(f"📈 Status: {'✅ GOOD' if quality['status'] == 'Good' else '⚠️ NEEDS IMPROVEMENT'}")

        print("\n📋 Detailed Metrics:")
        for metric, data in result['quality_scores'].items():
            if metric != 'overall':
                print(f"  • {metric.replace('_', ' ').title()}: {data['score']:.2f} ({data['status']})")

        print("\n🎨 Class Distribution:")
        for class_name, stats in result['class_distribution'].items():
            if stats['percentage'] > 1.0:  # Only show classes with >1%
                print(f"  • {class_name}: {stats['percentage']:.1f}% ({stats['pixels']:,} pixels)")

        # Recommendation
        if quality['score'] >= 0.6:
            print("\n✅ RECOMMENDATION: This segmentation quality is suitable for video processing!")
        else:
            print("\n⚠️ RECOMMENDATION: Consider improving the model before video processing.")
            print("   Possible improvements:")
            print("   • Try different preprocessing")
            print("   • Use fine-tuned model on similar data")
            print("   • Check image quality and lighting")

        print(f"\n💾 Results saved to: {checker.results_dir}")

        # Display images in Colab
        if COLAB_ENV:
            print("\n🖼️ Visual Results:")
            display(HTML(f"""
            <div style="display: flex; gap: 10px;">
                <div>
                    <h4>Original</h4>
                    <img src="{saved_files['image']}" style="max-width: 300px;">
                </div>
                <div>
                    <h4>Segmentation</h4>
                    <img src="{saved_files['segmented']}" style="max-width: 300px;">
                </div>
            </div>
            """))

    return result

# ===============================================================================
# STEP 6: MAIN INTERFACE
# ===============================================================================
def main_quality_check():
    """Main interface for quality checking"""
    print("🎯 SEGMENTATION QUALITY VALIDATION SYSTEM")
    print("=" * 70)
    print("This system helps you validate segmentation quality before video processing")
    print("\nOptions:")
    print("1. 🧪 Run complete validation with sample images")
    print("2. 📤 Test your own image")
    print("3. ℹ️  Show system information")
    print("0. 🚪 Exit")

    while True:
        try:
            choice = input("\n👉 Enter your choice (0-3): ").strip()

            if choice == '0':
                print("👋 Quality check completed!")
                break
            elif choice == '1':
                checker = SegmentationQualityChecker()
                success = checker.run_step_by_step_validation()
                if success:
                    print("\n🎉 Validation passed! You can proceed with video processing.")
                else:
                    print("\n⚠️ Consider model improvements before video processing.")
            elif choice == '2':
                test_user_image()
            elif choice == '3':
                print(f"\n💻 System Information:")
                print(f"  • Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")
                print(f"  • CUDA Available: {torch.cuda.is_available()}")
                if torch.cuda.is_available():
                    print(f"  • GPU: {torch.cuda.get_device_name()}")
                print(f"  • Environment: {'Google Colab' if COLAB_ENV else 'Local'}")
            else:
                print("❌ Invalid choice. Please try again.")

        except KeyboardInterrupt:
            print("\n⏹️ Interrupted by user")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# ===============================================================================
# EXECUTION
# ===============================================================================
if __name__ == "__main__":
    main_quality_check()

# ===============================================================================
# QUICK START INSTRUCTIONS
# ===============================================================================
"""
🚀 QUICK START GUIDE:

1. RUN THE CODE:
   - Execute this entire cell/script
   - It will initialize the quality checking system

2. CHOOSE OPTION 1 FOR COMPLETE VALIDATION:
   - Creates realistic traffic sample images
   - Tests segmentation quality on each sample
   - Provides detailed quality scores and grades
   - Gives clear recommendation (Ready/Not Ready for video)

3. CHOOSE OPTION 2 TO TEST YOUR OWN IMAGE:
   - Upload your traffic image
   - Get detailed quality analysis
   - See if it's suitable for video processing

4. INTERPRET RESULTS:
   - Quality Score 0.6+ = Good for video processing
   - Grade A/B = Excellent/Good quality
   - Grade C/D = Needs improvement

5. WHAT TO EXPECT:
   ✅ Good road surface detection
   ✅ Clear traffic element recognition
   ✅ Low fragmentation
   ✅ Coherent object boundaries

This validation ensures your model will work well on videos before processing!
"""

✅ Google Colab environment detected
🎯 SEGMENTATION QUALITY VALIDATION SYSTEM
This system helps you validate segmentation quality before video processing

Options:
1. 🧪 Run complete validation with sample images
2. 📤 Test your own image
3. ℹ️  Show system information
0. 🚪 Exit

👉 Enter your choice (0-3): 2

📤 USER IMAGE QUALITY TEST
🔄 Loading Segformer model...
✅ Model loaded successfully!
📱 Device: cpu
🎯 Classes: 19
📁 Please upload your traffic image:


Saving KakaoTalk_20250717_091440196_08.jpg to KakaoTalk_20250717_091440196_08 (1).jpg
🔍 Analyzing: KakaoTalk_20250717_091440196_08 (1).jpg

📊 QUALITY ANALYSIS RESULTS
🎯 Overall Quality Score: 0.79
🎓 Grade: B+
📈 Status: ✅ GOOD

📋 Detailed Metrics:
  • Road Coverage: 1.00 (Good)
  • Fragmentation: 1.00 (Good)
  • Traffic Detection: 1.00 (Good)
  • Spatial Coherence: 0.15 (Poor)

🎨 Class Distribution:
  • road: 25.6% (382,434 pixels)
  • building: 8.9% (132,861 pixels)
  • pole: 1.2% (17,321 pixels)
  • vegetation: 12.4% (184,783 pixels)
  • sky: 30.7% (457,928 pixels)
  • car: 19.9% (297,204 pixels)

✅ RECOMMENDATION: This segmentation quality is suitable for video processing!

💾 Results saved to: ./segmentation_quality_check/results

🖼️ Visual Results:



👉 Enter your choice (0-3): 0
👋 Quality check completed!


"\n🚀 QUICK START GUIDE:\n\n1. RUN THE CODE:\n   - Execute this entire cell/script\n   - It will initialize the quality checking system\n\n2. CHOOSE OPTION 1 FOR COMPLETE VALIDATION:\n   - Creates realistic traffic sample images\n   - Tests segmentation quality on each sample\n   - Provides detailed quality scores and grades\n   - Gives clear recommendation (Ready/Not Ready for video)\n\n3. CHOOSE OPTION 2 TO TEST YOUR OWN IMAGE:\n   - Upload your traffic image\n   - Get detailed quality analysis\n   - See if it's suitable for video processing\n\n4. INTERPRET RESULTS:\n   - Quality Score 0.6+ = Good for video processing\n   - Grade A/B = Excellent/Good quality\n   - Grade C/D = Needs improvement\n\n5. WHAT TO EXPECT:\n   ✅ Good road surface detection\n   ✅ Clear traffic element recognition\n   ✅ Low fragmentation\n   ✅ Coherent object boundaries\n\nThis validation ensures your model will work well on videos before processing!\n"

VIDEO PROCESSING

In [5]:
# ===============================================================================
# VIDEO PROCESSING - READY TO GO!
# ===============================================================================
# Your quality check PASSED with score 0.79 (Grade B+)
# Now let's process your video with the validated model!
# ===============================================================================

import torch
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
from datetime import datetime

# For file upload in Colab
try:
    from google.colab import files
    COLAB_ENV = True
    print("✅ Google Colab environment detected")
except ImportError:
    COLAB_ENV = False
    print("📱 Local environment detected")

class VideoSegmentationProcessor:
    """Ready-to-use video segmentation processor"""

    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.processor = None
        self.model = None
        self.output_dir = "./video_segmentation_output"
        os.makedirs(self.output_dir, exist_ok=True)

        # Color mapping (same as quality check)
        self.color_map = {
            0: [128, 64, 128],   # road - purple
            1: [244, 35, 232],   # sidewalk - pink
            2: [70, 70, 70],     # building - gray
            5: [153, 153, 153],  # pole - light gray
            6: [250, 170, 30],   # traffic_light - orange
            7: [220, 220, 0],    # traffic_sign - yellow
            8: [107, 142, 35],   # vegetation - olive
            10: [70, 130, 180],  # sky - steel blue
            11: [220, 20, 60],   # person - crimson
            13: [0, 0, 142],     # car - dark blue
            14: [0, 0, 70],      # truck - darker blue
            15: [0, 60, 100],    # bus - navy
            17: [0, 0, 230],     # motorcycle - blue
            18: [119, 11, 32],   # bicycle - dark red
        }

    def load_validated_model(self):
        """Load the same model that passed quality validation"""
        print("🚀 LOADING VALIDATED MODEL")
        print("=" * 50)
        print("Loading the same Cityscapes model that passed your quality check...")

        try:
            model_name = "nvidia/segformer-b0-finetuned-cityscapes-1024-1024"
            self.processor = SegformerImageProcessor.from_pretrained(model_name)
            self.model = SegformerForSemanticSegmentation.from_pretrained(model_name)
            self.model.to(self.device)
            self.model.eval()

            print("✅ Model loaded successfully!")
            print(f"📱 Device: {self.device}")
            print("🎯 Quality validated: Score 0.79 (Grade B+)")
            print("✅ Ready for video processing!")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            return False

    def predict_frame(self, frame_rgb):
        """Predict segmentation for a single frame"""
        # Convert to PIL Image
        image_pil = Image.fromarray(frame_rgb)

        # Process
        inputs = self.processor(image_pil, return_tensors="pt").to(self.device)

        # Predict
        with torch.no_grad():
            outputs = self.model(**inputs)
            predictions = torch.nn.functional.interpolate(
                outputs.logits,
                size=image_pil.size[::-1],
                mode="bilinear",
                align_corners=False,
            )
            predicted_map = predictions.squeeze().cpu().numpy().argmax(axis=0)

        return predicted_map

    def create_colored_overlay(self, prediction, alpha=0.6):
        """Create colored segmentation overlay"""
        h, w = prediction.shape
        colored_mask = np.zeros((h, w, 3), dtype=np.uint8)

        # Apply colors
        for class_id, color in self.color_map.items():
            mask = prediction == class_id
            colored_mask[mask] = color

        return colored_mask

    def upload_video(self):
        """Upload video file (Colab version)"""
        print("📤 VIDEO UPLOAD")
        print("=" * 30)

        if not COLAB_ENV:
            video_path = input("📁 Enter path to your video file: ").strip()
            if not os.path.exists(video_path):
                print(f"❌ Video file not found: {video_path}")
                return None
            return video_path

        print("📁 Please select your traffic video to upload:")
        uploaded = files.upload()

        if not uploaded:
            print("❌ No video uploaded!")
            return None

        video_filename = list(uploaded.keys())[0]
        print(f"✅ Uploaded: {video_filename}")
        return video_filename

    def process_video(self, video_path, max_frames=None, blend_alpha=0.6):
        """Process video with validated segmentation model"""
        print(f"\n🎬 PROCESSING VIDEO: {video_path}")
        print("=" * 60)

        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Error opening video: {video_path}")
            return None

        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if max_frames:
            total_frames = min(total_frames, max_frames)

        print(f"📹 Video Properties:")
        print(f"  • Resolution: {width}x{height}")
        print(f"  • FPS: {fps}")
        print(f"  • Frames to process: {total_frames:,}")
        print(f"  • Duration: {total_frames/fps:.1f} seconds")

        # Setup output video
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f"segmented_traffic_video_{timestamp}.mp4"
        output_path = os.path.join(self.output_dir, output_filename)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        print(f"💾 Output: {output_path}")
        print(f"🎨 Blend alpha: {blend_alpha} (0=mask only, 1=original only)")

        # Process frames
        print("\n⏳ Processing frames...")
        frame_count = 0
        start_time = time.time()

        with tqdm(total=total_frames, desc="🎬 Segmenting", unit="frames") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret or (max_frames and frame_count >= max_frames):
                    break

                # Convert BGR to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Predict segmentation
                prediction = self.predict_frame(frame_rgb)

                # Create colored overlay
                colored_mask = self.create_colored_overlay(prediction)

                # Blend with original frame
                blended = cv2.addWeighted(frame_rgb, blend_alpha, colored_mask, 1-blend_alpha, 0)

                # Convert back to BGR and write
                blended_bgr = cv2.cvtColor(blended, cv2.COLOR_RGB2BGR)
                out.write(blended_bgr)

                frame_count += 1
                pbar.update(1)

                # Update stats every 50 frames
                if frame_count % 50 == 0:
                    elapsed = time.time() - start_time
                    current_fps = frame_count / elapsed
                    remaining_frames = total_frames - frame_count
                    eta_seconds = remaining_frames / current_fps if current_fps > 0 else 0

                    pbar.set_postfix({
                        'FPS': f'{current_fps:.1f}',
                        'ETA': f'{eta_seconds/60:.1f}m'
                    })

        # Cleanup
        cap.release()
        out.release()

        # Final statistics
        total_time = time.time() - start_time
        avg_fps = frame_count / total_time

        print(f"\n✅ VIDEO PROCESSING COMPLETED!")
        print("=" * 50)
        print(f"📊 Processing Statistics:")
        print(f"  • Frames processed: {frame_count:,}")
        print(f"  • Total time: {total_time/60:.1f} minutes")
        print(f"  • Average FPS: {avg_fps:.1f}")
        print(f"  • Output saved: {output_path}")
        print(f"  • File size: {os.path.getsize(output_path)/1024/1024:.1f} MB")

        return output_path

# ===============================================================================
# MAIN VIDEO PROCESSING INTERFACE
# ===============================================================================
def main_video_processing():
    """Main interface for video processing"""
    print("🎬 VIDEO SEGMENTATION - QUALITY VALIDATED!")
    print("=" * 60)
    print("✅ Your model passed quality check with score 0.79 (Grade B+)")
    print("🚀 Ready to process your traffic video!")

    # Initialize processor
    processor = VideoSegmentationProcessor()

    # Load model
    if not processor.load_validated_model():
        print("❌ Failed to load model!")
        return

    print("\n🎯 PROCESSING OPTIONS:")
    print("=" * 30)
    print("1. 📤 Upload and process video (full)")
    print("2. 📹 Process video from path")
    print("3. 🧪 Process video (first 100 frames only - for testing)")
    print("4. ⚙️  Custom processing options")
    print("0. 🚪 Exit")

    while True:
        try:
            choice = input("\n👉 Enter your choice (0-4): ").strip()

            if choice == '0':
                print("👋 Video processing session ended!")
                break

            elif choice == '1':
                # Upload and process full video
                video_path = processor.upload_video()
                if video_path:
                    output_path = processor.process_video(video_path)
                    if output_path:
                        print(f"\n🎉 SUCCESS! Your segmented video is ready!")
                        print(f"📁 Location: {output_path}")

            elif choice == '2':
                # Process from path
                video_path = input("📁 Enter video file path: ").strip()
                if video_path and os.path.exists(video_path):
                    output_path = processor.process_video(video_path)
                    if output_path:
                        print(f"\n🎉 SUCCESS! Your segmented video is ready!")
                        print(f"📁 Location: {output_path}")
                else:
                    print("❌ Video file not found!")

            elif choice == '3':
                # Test with first 100 frames
                video_path = processor.upload_video() if COLAB_ENV else input("📁 Enter video path: ").strip()
                if video_path:
                    print("🧪 Processing first 100 frames for testing...")
                    output_path = processor.process_video(video_path, max_frames=100)
                    if output_path:
                        print(f"\n🎉 TEST COMPLETE! Sample video created!")
                        print(f"📁 Location: {output_path}")

            elif choice == '4':
                # Custom options
                video_path = input("📁 Enter video path: ").strip()
                if not os.path.exists(video_path):
                    print("❌ Video file not found!")
                    continue

                try:
                    max_frames = input("🎬 Max frames (or press Enter for all): ").strip()
                    max_frames = int(max_frames) if max_frames else None

                    blend_alpha = input("🎨 Blend ratio 0.0-1.0 (0.6 default): ").strip()
                    blend_alpha = float(blend_alpha) if blend_alpha else 0.6

                    output_path = processor.process_video(video_path, max_frames, blend_alpha)
                    if output_path:
                        print(f"\n🎉 SUCCESS! Custom processed video ready!")
                        print(f"📁 Location: {output_path}")

                except ValueError:
                    print("❌ Invalid input values!")

            else:
                print("❌ Invalid choice. Please try again.")

        except KeyboardInterrupt:
            print("\n⏹️ Operation cancelled by user")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# ===============================================================================
# QUICK START EXECUTION
# ===============================================================================
if __name__ == "__main__":
    print("🎉 CONGRATULATIONS!")
    print("Your segmentation model passed quality validation!")
    print("Score: 0.79 (Grade B+) ✅")
    print("\nStarting video processing...")
    print("=" * 60)

    main_video_processing()

# ===============================================================================
# USAGE SUMMARY
# ===============================================================================
"""
🎯 SUMMARY - YOU'RE READY!

✅ QUALITY CHECK PASSED: Score 0.79 (Grade B+)
✅ MODEL VALIDATED: Much better than your previous YOLO approach
✅ READY FOR VIDEO: No more fragmented, noisy results

RECOMMENDED WORKFLOW:
1. Choose option 3 first (test with 100 frames)
2. Check the sample output quality
3. If satisfied, process your full video with option 1 or 2

EXPECTED IMPROVEMENTS:
✅ Clean road surface detection (no more blue noise)
✅ Proper traffic light/sign recognition
✅ Coherent vehicle boundaries
✅ Realistic background segmentation

Your video processing should now give you the clean,
professional results you were looking for! 🚀
"""

✅ Google Colab environment detected
🎉 CONGRATULATIONS!
Your segmentation model passed quality validation!
Score: 0.79 (Grade B+) ✅

Starting video processing...
🎬 VIDEO SEGMENTATION - QUALITY VALIDATED!
✅ Your model passed quality check with score 0.79 (Grade B+)
🚀 Ready to process your traffic video!
🚀 LOADING VALIDATED MODEL
Loading the same Cityscapes model that passed your quality check...
✅ Model loaded successfully!
📱 Device: cpu
🎯 Quality validated: Score 0.79 (Grade B+)
✅ Ready for video processing!

🎯 PROCESSING OPTIONS:
1. 📤 Upload and process video (full)
2. 📹 Process video from path
3. 🧪 Process video (first 100 frames only - for testing)
4. ⚙️  Custom processing options
0. 🚪 Exit

👉 Enter your choice (0-4): 1
📤 VIDEO UPLOAD
📁 Please select your traffic video to upload:


Saving Sunny day.mp4 to Sunny day.mp4
✅ Uploaded: Sunny day.mp4

🎬 PROCESSING VIDEO: Sunny day.mp4
📹 Video Properties:
  • Resolution: 1600x720
  • FPS: 24
  • Frames to process: 2,226
  • Duration: 92.8 seconds
💾 Output: ./video_segmentation_output/segmented_traffic_video_20250816_021323.mp4
🎨 Blend alpha: 0.6 (0=mask only, 1=original only)

⏳ Processing frames...


🎬 Segmenting: 100%|██████████| 2226/2226 [46:27<00:00,  1.25s/frames, FPS=0.8, ETA=0.5m]



✅ VIDEO PROCESSING COMPLETED!
📊 Processing Statistics:
  • Frames processed: 2,226
  • Total time: 46.5 minutes
  • Average FPS: 0.8
  • Output saved: ./video_segmentation_output/segmented_traffic_video_20250816_021323.mp4
  • File size: 99.8 MB

🎉 SUCCESS! Your segmented video is ready!
📁 Location: ./video_segmentation_output/segmented_traffic_video_20250816_021323.mp4

👉 Enter your choice (0-4): 0
👋 Video processing session ended!


"\n🎯 SUMMARY - YOU'RE READY!\n\n✅ QUALITY CHECK PASSED: Score 0.79 (Grade B+)\n✅ MODEL VALIDATED: Much better than your previous YOLO approach\n✅ READY FOR VIDEO: No more fragmented, noisy results\n\nRECOMMENDED WORKFLOW:\n1. Choose option 3 first (test with 100 frames)\n2. Check the sample output quality\n3. If satisfied, process your full video with option 1 or 2\n\nEXPECTED IMPROVEMENTS:\n✅ Clean road surface detection (no more blue noise)\n✅ Proper traffic light/sign recognition\n✅ Coherent vehicle boundaries  \n✅ Realistic background segmentation\n\nYour video processing should now give you the clean, \nprofessional results you were looking for! 🚀\n"

Model Traing with labeling Data

In [ ]:
"""
YOLO Training Tool - Multi-Version Support with Fixed Dataset and File Handling

Supports YOLOv8, YOLOv9, YOLOv10, and YOLO11 with automatic version detection.

For Jupyter/Colab users, use the simple function:
    model_path = train_yolo_simple("/path/to/data.zip", classes="all", epochs=100)
    model_path = train_yolo_simple("/path/to/data.zip", classes="0,2,5", epochs=50, model="yolov8n.pt")
    model_path = train_yolo_simple("/path/to/data.zip", model="yolov10m.pt", yolo_version="yolov10")

For command line usage:
    python yolo11_trainer.py --cli
    python yolo11_trainer.py --zip data.zip --classes all --epochs 100 --model yolov8s.pt
"""

import os
import sys
import zipfile
import json
import yaml
from pathlib import Path
import threading
import shutil
import argparse
import subprocess
import random
import time
import warnings

# Suppress common warnings that clutter the output
warnings.filterwarnings("ignore", category=UserWarning, module="torch.*")
warnings.filterwarnings("ignore", category=FutureWarning, module="torch.*")

# These will be imported after checking if they're available
# import torch
# from ultralytics import YOLO

# Try to import tkinter, handle if no display available
GUI_AVAILABLE = True
try:
    import tkinter as tk
    from tkinter import ttk, filedialog, messagebox, scrolledtext
    # Test if display is available
    root_test = tk.Tk()
    root_test.withdraw()
    root_test.destroy()
except (ImportError, tk.TclError) as e:
    GUI_AVAILABLE = False
    print("=" * 60)
    print("YOLO11 Training Tool")
    print("=" * 60)
    print(f"GUI not available: {e}")
    print("Running in command-line mode...")
    print("\nTo enable GUI:")
    print("- On Linux/WSL: Install X server (Xming, VcXsrv, or X11)")
    print("- On SSH: Use 'ssh -X' for X11 forwarding")
    print("- On headless servers: Use CLI mode with --cli flag")
    print("=" * 60)

def check_and_install_packages():
    """Check and install required packages with better error handling"""
    missing_packages = []
    installation_commands = []

    print("🔍 Checking required packages...")

    # Check PyTorch
    try:
        import torch
        print(f"✅ PyTorch {torch.__version__} is available")

        # Check CUDA availability with better error handling
        if torch.cuda.is_available():
            gpu_count = torch.cuda.device_count()
            print(f"✅ CUDA available with {gpu_count} GPU(s)")
            for i in range(gpu_count):
                try:
                    gpu_name = torch.cuda.get_device_name(i)
                    print(f"   GPU {i}: {gpu_name}")
                except Exception as e:
                    print(f"   GPU {i}: Unknown (error: {e})")
        else:
            print("⚠️  CUDA not available - will use CPU")

    except ImportError:
        print("❌ PyTorch not found")
        missing_packages.append("torch")
        installation_commands.append("pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124")

    # Check Ultralytics
    try:
        import ultralytics
        print(f"✅ Ultralytics {ultralytics.__version__} is available")
    except ImportError:
        print("❌ Ultralytics not found")
        missing_packages.append("ultralytics")
        installation_commands.append("pip install ultralytics")

    # Install missing packages
    if missing_packages:
        print(f"\n📦 Installing {len(missing_packages)} missing package(s)...")

        for i, cmd in enumerate(installation_commands):
            package_name = missing_packages[i]
            print(f"\n⏳ Installing {package_name}...")

            try:
                # Use subprocess with better error handling
                result = subprocess.run(
                    cmd.split(),
                    capture_output=True,
                    text=True,
                    timeout=300  # 5 minute timeout
                )

                if result.returncode == 0:
                    print(f"✅ {package_name} installed successfully")
                else:
                    print(f"❌ Failed to install {package_name}")
                    print(f"Error: {result.stderr}")
                    return False

            except subprocess.TimeoutExpired:
                print(f"❌ Installation of {package_name} timed out")
                return False
            except Exception as e:
                print(f"❌ Error installing {package_name}: {e}")
                return False

        print("\n🔄 Reloading modules...")
        # Try to import again after installation
        try:
            import torch
            from ultralytics import YOLO
            print("✅ All packages loaded successfully")
        except ImportError as e:
            print(f"❌ Still missing packages after installation: {e}")
            return False

    return True

class FileHandler:
    """Handles file operations with improved error handling"""

    @staticmethod
    def normalize_path(file_path):
        """Normalize and clean file path"""
        if not file_path:
            return ""

        # Remove quotes and whitespace
        file_path = file_path.strip().strip('"\'')

        # Handle different path formats
        if file_path.startswith('\\\\'):
            # UNC path
            return file_path

        # Normalize path separators
        file_path = os.path.normpath(file_path)

        # Convert to absolute path if relative
        if not os.path.isabs(file_path):
            file_path = os.path.abspath(file_path)

        return file_path

    @staticmethod
    def check_file_access(file_path):
        """Check if file exists and is accessible"""
        try:
            # Check existence
            if not os.path.exists(file_path):
                return False, f"File does not exist: {file_path}"

            # Check if it's a file (not directory)
            if not os.path.isfile(file_path):
                return False, f"Path is not a file: {file_path}"

            # Check read permissions
            if not os.access(file_path, os.R_OK):
                return False, f"No read permission for file: {file_path}"

            # Check file size
            file_size = os.path.getsize(file_path)
            if file_size == 0:
                return False, f"File is empty: {file_path}"

            return True, f"File accessible, size: {file_size:,} bytes"

        except Exception as e:
            return False, f"Error checking file: {str(e)}"

    @staticmethod
    def is_valid_zip_detailed(file_path):
        """Check if file is a valid ZIP archive with detailed reporting"""
        try:
            # Basic file checks first
            accessible, msg = FileHandler.check_file_access(file_path)
            if not accessible:
                return False, msg

            print(f"📁 Checking ZIP file: {os.path.basename(file_path)}")
            print(f"   Path: {file_path}")
            print(f"   {msg}")

            # Try to read as ZIP
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    # Get file list
                    file_list = zip_ref.namelist()
                    if not file_list:
                        return False, "ZIP file is empty"

                    print(f"   📋 Contains {len(file_list)} files/folders")

                    # Test ZIP integrity
                    print("   🔍 Testing ZIP integrity...")
                    bad_file = zip_ref.testzip()
                    if bad_file:
                        return False, f"ZIP contains corrupted file: {bad_file}"

                    # Count relevant files
                    image_files = []
                    label_files = []
                    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp'}

                    for filename in file_list:
                        if filename.lower().endswith(tuple(image_extensions)):
                            image_files.append(filename)
                        elif filename.lower().endswith('.txt') and not filename.lower().endswith('classes.txt'):
                            label_files.append(filename)

                    print(f"   🖼️  Found {len(image_files)} image files")
                    print(f"   🏷️  Found {len(label_files)} label files")

                    if len(image_files) == 0:
                        return False, "No image files found in ZIP"

                    return True, f"Valid ZIP with {len(image_files)} images and {len(label_files)} labels"

            except zipfile.BadZipFile:
                return False, "File is not a valid ZIP archive"
            except Exception as e:
                return False, f"Error reading ZIP file: {str(e)}"

        except Exception as e:
            return False, f"Unexpected error: {str(e)}"

    @staticmethod
    def find_files_interactive():
        """Interactive file finder with suggestions"""
        print("\n🔍 File Path Helper")
        print("Let's find your ZIP file step by step...")

        # Check current directory
        current_dir = os.getcwd()
        print(f"\n📁 Current directory: {current_dir}")

        # Look for ZIP files in current directory
        zip_files = []
        try:
            for file in os.listdir(current_dir):
                if file.lower().endswith('.zip'):
                    zip_files.append(file)
        except Exception:
            pass

        if zip_files:
            print(f"🎯 Found {len(zip_files)} ZIP file(s) in current directory:")
            for i, file in enumerate(zip_files):
                file_path = os.path.join(current_dir, file)
                size = os.path.getsize(file_path)
                print(f"   {i+1}. {file} ({size:,} bytes)")

            try:
                choice = input(f"\nEnter number (1-{len(zip_files)}) to select, or 'c' to continue with custom path: ").strip()
                if choice.isdigit() and 1 <= int(choice) <= len(zip_files):
                    selected_file = os.path.join(current_dir, zip_files[int(choice)-1])
                    print(f"✅ Selected: {selected_file}")
                    return selected_file
            except KeyboardInterrupt:
                raise
            except Exception:
                pass

        # Check common directories
        common_dirs = []
        home_dir = os.path.expanduser("~")
        if os.path.exists(home_dir):
            common_dirs.append(("Home", home_dir))

            # Check Downloads folder
            downloads_dir = os.path.join(home_dir, "Downloads")
            if os.path.exists(downloads_dir):
                common_dirs.append(("Downloads", downloads_dir))

            # Check Desktop
            desktop_dir = os.path.join(home_dir, "Desktop")
            if os.path.exists(desktop_dir):
                common_dirs.append(("Desktop", desktop_dir))

            # Check OneDrive Desktop (Windows)
            onedrive_desktop = os.path.join(home_dir, "OneDrive", "바탕 화면")
            if os.path.exists(onedrive_desktop):
                common_dirs.append(("OneDrive Desktop", onedrive_desktop))

        # Check for ZIP files in common directories
        for dir_name, dir_path in common_dirs:
            try:
                zip_files_in_dir = [f for f in os.listdir(dir_path) if f.lower().endswith('.zip')]
                if zip_files_in_dir:
                    print(f"\n📂 Found ZIP files in {dir_name} ({dir_path}):")
                    for file in zip_files_in_dir[:5]:  # Show first 5
                        file_path = os.path.join(dir_path, file)
                        try:
                            size = os.path.getsize(file_path)
                            print(f"   • {file} ({size:,} bytes)")
                        except Exception:
                            print(f"   • {file}")
                    if len(zip_files_in_dir) > 5:
                        print(f"   ... and {len(zip_files_in_dir) - 5} more")
            except Exception:
                continue

        print(f"\n💡 Tips for entering file path:")
        print(f"   • Use forward slashes (/) or double backslashes (\\\\)")
        print(f"   • Drag and drop the file into terminal (if supported)")
        print(f"   • Use quotes if path contains spaces")
        print(f"   • Use Tab for auto-completion (if supported)")

        return None

class DatasetManager:
    """Handles all dataset operations with robust error handling"""

    def __init__(self, log_func=print):
        self.log = log_func
        self.image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp'}

    def analyze_dataset_structure(self, extract_path):
        """Analyze and report dataset structure with detailed logging"""
        self.log("🔍 Analyzing dataset structure...")

        structure_info = {
            'images': [],
            'labels': [],
            'image_dirs': {},
            'label_dirs': {},
            'total_images': 0,
            'total_labels': 0,
            'class_ids': set()
        }

        # Walk through all directories
        for root, dirs, files in os.walk(extract_path):
            rel_root = os.path.relpath(root, extract_path)
            if rel_root == '.':
                rel_root = 'root'

            # Count images and labels in this directory
            images_in_dir = []
            labels_in_dir = []

            for file in files:
                file_lower = file.lower()
                if any(file_lower.endswith(ext) for ext in self.image_extensions):
                    images_in_dir.append(file)
                    structure_info['images'].append(os.path.join(root, file))
                elif file_lower.endswith('.txt') and file_lower not in ['classes.txt', 'readme.txt']:
                    labels_in_dir.append(file)
                    structure_info['labels'].append(os.path.join(root, file))

                    # Extract class IDs from this label file
                    try:
                        with open(os.path.join(root, file), 'r') as f:
                            for line in f:
                                if line.strip():
                                    parts = line.split()
                                    if len(parts) >= 5:
                                        try:
                                            class_id = int(parts[0])
                                            structure_info['class_ids'].add(class_id)
                                        except ValueError:
                                            continue
                    except Exception:
                        pass

            if images_in_dir:
                structure_info['image_dirs'][rel_root] = len(images_in_dir)
                structure_info['total_images'] += len(images_in_dir)

            if labels_in_dir:
                structure_info['label_dirs'][rel_root] = len(labels_in_dir)
                structure_info['total_labels'] += len(labels_in_dir)

        # Report findings
        self.log(f"📊 Dataset Analysis Complete:")
        self.log(f"   🖼️  Total images: {structure_info['total_images']}")
        self.log(f"   🏷️  Total labels: {structure_info['total_labels']}")
        self.log(f"   🎯 Unique classes: {len(structure_info['class_ids'])}")

        if structure_info['image_dirs']:
            self.log(f"   📁 Image directories:")
            for dir_name, count in structure_info['image_dirs'].items():
                self.log(f"      {dir_name}: {count} images")

        if structure_info['label_dirs']:
            self.log(f"   📂 Label directories:")
            for dir_name, count in structure_info['label_dirs'].items():
                self.log(f"      {dir_name}: {count} labels")

        return structure_info

    def create_yolo_structure(self, base_path):
        """Create YOLO directory structure"""
        yolo_dirs = {
            'train_images': os.path.join(base_path, 'train', 'images'),
            'train_labels': os.path.join(base_path, 'train', 'labels'),
            'val_images': os.path.join(base_path, 'val', 'images'),
            'val_labels': os.path.join(base_path, 'val', 'labels'),
            'test_images': os.path.join(base_path, 'test', 'images'),
            'test_labels': os.path.join(base_path, 'test', 'labels')
        }

        # Create all directories
        for dir_name, dir_path in yolo_dirs.items():
            os.makedirs(dir_path, exist_ok=True)
            self.log(f"   📁 Created: {os.path.relpath(dir_path, base_path)}")

        return yolo_dirs

    def find_image_label_pairs(self, structure_info):
        """Find matching image-label pairs"""
        self.log("🔍 Finding image-label pairs...")

        image_label_pairs = []
        unmatched_images = []

        for img_path in structure_info['images']:
            img_name = os.path.basename(img_path)
            img_name_no_ext = os.path.splitext(img_name)[0]

            # Look for corresponding label file
            label_path = None
            for lbl_path in structure_info['labels']:
                lbl_name = os.path.basename(lbl_path)
                lbl_name_no_ext = os.path.splitext(lbl_name)[0]

                if img_name_no_ext == lbl_name_no_ext:
                    label_path = lbl_path
                    break

            if label_path and os.path.exists(label_path):
                image_label_pairs.append((img_path, label_path, img_name))
            else:
                unmatched_images.append(img_name)

        self.log(f"✅ Found {len(image_label_pairs)} valid image-label pairs")
        if unmatched_images:
            self.log(f"⚠️  {len(unmatched_images)} images without labels")
            if len(unmatched_images) <= 10:
                for img in unmatched_images[:10]:
                    self.log(f"      {img}")
            else:
                for img in unmatched_images[:5]:
                    self.log(f"      {img}")
                self.log(f"      ... and {len(unmatched_images) - 5} more")

        return image_label_pairs

    def split_dataset(self, image_label_pairs, train_ratio=0.7, val_ratio=0.2):
        """Split dataset into train/val/test with minimum validation guarantee"""
        if len(image_label_pairs) == 0:
            return {'train': [], 'val': [], 'test': []}

        # Shuffle for random split
        random.shuffle(image_label_pairs)
        total_pairs = len(image_label_pairs)

        # Ensure minimum validation set size
        min_val_size = max(1, min(10, total_pairs // 10))  # At least 1, max 10, or 10% of dataset

        if total_pairs < 3:
            # Very small dataset - put most in training, at least 1 in validation
            if total_pairs == 1:
                splits = {'train': image_label_pairs, 'val': [], 'test': []}
            elif total_pairs == 2:
                splits = {'train': image_label_pairs[:1], 'val': image_label_pairs[1:], 'test': []}
            else:  # total_pairs == 3
                splits = {'train': image_label_pairs[:2], 'val': image_label_pairs[2:], 'test': []}
        else:
            # Calculate split points
            val_size = max(min_val_size, int(total_pairs * val_ratio))
            test_size = max(1, int(total_pairs * (1 - train_ratio - val_ratio)))
            train_size = total_pairs - val_size - test_size

            # Ensure train_size is positive
            if train_size < 1:
                train_size = total_pairs - val_size
                test_size = 0

            train_end = train_size
            val_end = train_size + val_size

            splits = {
                'train': image_label_pairs[:train_end],
                'val': image_label_pairs[train_end:val_end],
                'test': image_label_pairs[val_end:] if test_size > 0 else []
            }

        self.log(f"📊 Dataset split:")
        self.log(f"   🏋️  Training: {len(splits['train'])} samples ({len(splits['train'])/total_pairs*100:.1f}%)")
        self.log(f"   ✅ Validation: {len(splits['val'])} samples ({len(splits['val'])/total_pairs*100:.1f}%)")
        if splits['test']:
            self.log(f"   🧪 Test: {len(splits['test'])} samples ({len(splits['test'])/total_pairs*100:.1f}%)")

        return splits

    def copy_files_to_splits(self, splits, yolo_dirs):
        """Copy files to train/val/test directories"""
        self.log("📋 Copying files to YOLO structure...")

        total_copied = 0

        for split_name, pairs in splits.items():
            if len(pairs) == 0:
                continue

            img_dir = yolo_dirs[f'{split_name}_images']
            lbl_dir = yolo_dirs[f'{split_name}_labels']

            split_copied = 0

            for img_path, lbl_path, img_name in pairs:
                try:
                    # Copy image
                    if os.path.exists(img_path):
                        shutil.copy2(img_path, img_dir)
                        split_copied += 1
                        total_copied += 1
                    else:
                        self.log(f"⚠️  Image not found: {img_path}")
                        continue

                    # Copy label
                    if lbl_path and os.path.exists(lbl_path):
                        shutil.copy2(lbl_path, lbl_dir)
                    else:
                        self.log(f"⚠️  Label not found for: {img_name}")

                except Exception as e:
                    self.log(f"❌ Error copying {img_name}: {e}")
                    continue

            self.log(f"   {split_name}: {split_copied} files copied")

        return total_copied > 0

    def reorganize_dataset(self, extract_path, structure_info):
        """Complete dataset reorganization with robust error handling"""
        self.log("🔄 Reorganizing dataset to YOLO format...")

        # Create YOLO directory structure
        yolo_dirs = self.create_yolo_structure(extract_path)

        # Find image-label pairs
        image_label_pairs = self.find_image_label_pairs(structure_info)

        if len(image_label_pairs) == 0:
            self.log("❌ No valid image-label pairs found!")
            return False

        # Check if dataset already has splits
        has_existing_splits = self.check_existing_splits(structure_info)

        if has_existing_splits:
            self.log("✅ Preserving existing train/val splits")
            success = self.preserve_existing_splits(structure_info, yolo_dirs)
        else:
            self.log("🔀 Creating new train/val/test splits")
            splits = self.split_dataset(image_label_pairs)
            success = self.copy_files_to_splits(splits, yolo_dirs)

        if not success:
            return False

        # Verify the reorganization
        return self.verify_dataset_structure(yolo_dirs)

    def check_existing_splits(self, structure_info):
        """Check if dataset already has train/val directory structure"""
        has_train = any('train' in dir_name.lower() for dir_name in structure_info['image_dirs'].keys())
        has_val = any('val' in dir_name.lower() or 'valid' in dir_name.lower() for dir_name in structure_info['image_dirs'].keys())
        return has_train and has_val

    def preserve_existing_splits(self, structure_info, yolo_dirs):
        """Preserve existing train/val/test splits"""
        files_copied = 0

        for img_path in structure_info['images']:
            img_name = os.path.basename(img_path)
            img_name_no_ext = os.path.splitext(img_name)[0]

            if not os.path.exists(img_path):
                continue

            # Determine split based on directory path
            dir_path = os.path.dirname(img_path).lower()

            if 'train' in dir_path:
                dest_img_dir = yolo_dirs['train_images']
                dest_lbl_dir = yolo_dirs['train_labels']
            elif 'val' in dir_path or 'valid' in dir_path:
                dest_img_dir = yolo_dirs['val_images']
                dest_lbl_dir = yolo_dirs['val_labels']
            elif 'test' in dir_path:
                dest_img_dir = yolo_dirs['test_images']
                dest_lbl_dir = yolo_dirs['test_labels']
            else:
                # Default to train if unclear
                dest_img_dir = yolo_dirs['train_images']
                dest_lbl_dir = yolo_dirs['train_labels']

            try:
                # Copy image
                shutil.copy2(img_path, dest_img_dir)
                files_copied += 1

                # Find and copy corresponding label
                for lbl_path in structure_info['labels']:
                    lbl_name = os.path.basename(lbl_path)
                    lbl_name_no_ext = os.path.splitext(lbl_name)[0]

                    if img_name_no_ext == lbl_name_no_ext:
                        if os.path.exists(lbl_path):
                            shutil.copy2(lbl_path, dest_lbl_dir)
                        break

            except Exception as e:
                self.log(f"❌ Error copying {img_name}: {e}")
                continue

        self.log(f"✅ Copied {files_copied} files preserving splits")
        return files_copied > 0

    def verify_dataset_structure(self, yolo_dirs):
        """Verify that the dataset structure is correct"""
        self.log("🔍 Verifying dataset structure...")

        required_dirs = ['train_images', 'train_labels', 'val_images', 'val_labels']

        for dir_name in required_dirs:
            dir_path = yolo_dirs[dir_name]

            if not os.path.exists(dir_path):
                self.log(f"❌ Missing directory: {dir_path}")
                return False

            # Count files
            if 'images' in dir_name:
                files = [f for f in os.listdir(dir_path) if f.lower().endswith(tuple(self.image_extensions))]
            else:
                files = [f for f in os.listdir(dir_path) if f.endswith('.txt')]

            file_count = len(files)
            self.log(f"   ✅ {dir_name}: {file_count} files")

            # Check for empty critical directories
            if file_count == 0:
                if dir_name in ['train_images', 'train_labels']:
                    self.log(f"❌ Critical directory is empty: {dir_name}")
                    return False
                elif dir_name in ['val_images', 'val_labels']:
                    self.log(f"⚠️  Validation directory is empty: {dir_name}")
                    # Try to create validation set from training data
                    return self.create_validation_from_training(yolo_dirs)

        self.log("✅ Dataset structure verification passed")
        return True

    def create_validation_from_training(self, yolo_dirs):
        """Create validation set from training data when validation is empty"""
        self.log("🔄 Creating validation set from training data...")

        train_images_dir = yolo_dirs['train_images']
        train_labels_dir = yolo_dirs['train_labels']
        val_images_dir = yolo_dirs['val_images']
        val_labels_dir = yolo_dirs['val_labels']

        # Get all training images
        train_images = [f for f in os.listdir(train_images_dir) if f.lower().endswith(tuple(self.image_extensions))]

        if len(train_images) < 2:
            self.log("❌ Not enough training images to create validation set")
            return False

        # Move 20% of training to validation (minimum 1, maximum 20)
        val_count = max(1, min(20, len(train_images) // 5))

        # Randomly select images for validation
        random.shuffle(train_images)
        val_images = train_images[:val_count]

        self.log(f"📦 Moving {len(val_images)} samples to validation...")

        moved_count = 0
        for img_file in val_images:
            img_name_no_ext = os.path.splitext(img_file)[0]

            # Move image
            src_img = os.path.join(train_images_dir, img_file)
            dst_img = os.path.join(val_images_dir, img_file)

            if os.path.exists(src_img):
                shutil.move(src_img, dst_img)
                moved_count += 1

                # Move corresponding label if exists
                label_file = img_name_no_ext + '.txt'
                src_label = os.path.join(train_labels_dir, label_file)
                dst_label = os.path.join(val_labels_dir, label_file)

                if os.path.exists(src_label):
                    shutil.move(src_label, dst_label)

        self.log(f"✅ Created validation set with {moved_count} samples")
        return moved_count > 0

class YOLO11TrainerCLI:
    """Command-line interface for YOLO11 training with improved file handling"""

    def __init__(self):
        self.all_classes = []
        self.data_path = ""
        self.dataset_manager = DatasetManager(print)
        self.file_handler = FileHandler()
        self.check_gpu()

    def check_gpu(self):
        """Check for available GPUs with better error handling"""
        try:
            import torch
        except ImportError:
            print("⚠️ PyTorch not available, cannot check GPU status")
            return

        try:
            if torch.cuda.is_available():
                gpu_count = torch.cuda.device_count()
                gpu_names = []
                for i in range(gpu_count):
                    try:
                        gpu_names.append(torch.cuda.get_device_name(i))
                    except Exception:
                        gpu_names.append(f"GPU {i}")

                print(f"✅ {gpu_count} GPU(s) available: {', '.join(gpu_names)}")
                os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, range(gpu_count)))
            else:
                print("⚠️ No GPU available, will use CPU")
        except Exception as e:
            print(f"⚠️ GPU check failed: {e}")

    def get_zip_file_interactive(self):
        """Interactive ZIP file selection with better error handling"""
        max_attempts = 5
        attempt = 0

        while attempt < max_attempts:
            attempt += 1
            print(f"\n📁 Training Data (Attempt {attempt}/{max_attempts}):")

            if attempt == 1:
                # First attempt - just ask for path
                zip_path = input("Enter path to ZIP file: ").strip()
            elif attempt == 2:
                # Second attempt - provide help
                print("💡 Let me help you find the file...")
                suggested_file = self.file_handler.find_files_interactive()
                if suggested_file:
                    zip_path = suggested_file
                else:
                    zip_path = input("\nEnter full path to ZIP file: ").strip()
            else:
                # Subsequent attempts - more guidance
                print("🔧 Troubleshooting mode:")
                print("1. Make sure the file exists")
                print("2. Check the file is actually a ZIP file")
                print("3. Use the full path to the file")
                print("4. Make sure you have read permissions")
                zip_path = input("\nTry again with full path: ").strip()

            if not zip_path:
                print("❌ No path provided")
                continue

            # Normalize the path
            zip_path = self.file_handler.normalize_path(zip_path)
            print(f"🔍 Checking: {zip_path}")

            # Check if file is valid ZIP
            is_valid, message = self.file_handler.is_valid_zip_detailed(zip_path)

            if is_valid:
                print(f"✅ {message}")
                return zip_path
            else:
                print(f"❌ {message}")

                if "does not exist" in message.lower():
                    print("🔍 File not found. Please check:")
                    print(f"   • Path spelling: {zip_path}")
                    print(f"   • File location")
                    print(f"   • File permissions")
                elif "not a valid zip" in message.lower():
                    print("🔍 File format issue. Please check:")
                    print(f"   • File is actually a ZIP file")
                    print(f"   • File is not corrupted")
                    print(f"   • File downloaded completely")

                # On last attempt, give option to continue with CLI help
                if attempt == max_attempts:
                    print(f"\n🆘 Unable to load ZIP file after {max_attempts} attempts")
                    print("Would you like to:")
                    print("1. Try manual file selection")
                    print("2. Exit and fix the file issue")

                    choice = input("Enter choice (1/2): ").strip()
                    if choice == "1":
                        # Reset attempts for one more try
                        attempt = 0
                        max_attempts = 2
                        continue
                    else:
                        return None

        return None

    def load_data(self, zip_path):
        """Load and analyze the ZIP file with comprehensive error handling"""
        self.data_path = zip_path

        # Validate ZIP file first
        is_valid, message = self.file_handler.is_valid_zip_detailed(zip_path)
        if not is_valid:
            print(f"❌ {message}")
            return False

        print(f"✅ {message}")

        try:
            print("🚀 Starting data loading process...")

            # Extract ZIP file
            extract_path = "./temp_data"
            if os.path.exists(extract_path):
                print("🧹 Cleaning up previous data...")
                shutil.rmtree(extract_path)
            os.makedirs(extract_path)

            print("📦 Extracting ZIP file...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)

            print("✅ ZIP file extracted successfully")

            # Analyze dataset structure
            structure_info = self.dataset_manager.analyze_dataset_structure(extract_path)

            if structure_info['total_images'] == 0:
                print("❌ No image files found in dataset")
                print("💡 Make sure your ZIP contains image files (.jpg, .png, etc.)")
                return False

            if structure_info['total_labels'] == 0:
                print("❌ No label files found in dataset")
                print("💡 Make sure your ZIP contains YOLO format label files (.txt)")
                return False

            # Reorganize dataset
            if not self.dataset_manager.reorganize_dataset(extract_path, structure_info):
                print("❌ Failed to reorganize dataset")
                return False

            # Extract class information
            all_class_ids = sorted(list(structure_info['class_ids']))
            if not all_class_ids:
                print("❌ No valid class labels found")
                return False

            # Load class names
            self.all_classes = self.load_class_names(extract_path, all_class_ids)

            print(f"✅ Successfully loaded dataset with {len(self.all_classes)} classes:")
            for i, class_name in enumerate(self.all_classes):
                print(f"   {all_class_ids[i]:2d}: {class_name}")

            return True

        except Exception as e:
            print(f"❌ Failed to load data: {str(e)}")
            print("\n🔧 Troubleshooting tips:")
            print("• Make sure the ZIP file is not corrupted")
            print("• Check available disk space")
            print("• Verify file permissions")
            print("• Try extracting the ZIP manually first")
            return False

    def load_class_names(self, extract_path, all_class_ids):
        """Load class names from various sources"""
        class_names = [f"Class_{i}" for i in all_class_ids]

        for root, dirs, files in os.walk(extract_path):
            if 'classes.txt' in files:
                try:
                    with open(os.path.join(root, 'classes.txt'), 'r') as f:
                        names = [line.strip() for line in f if line.strip()]
                        if len(names) >= max(all_class_ids) + 1:
                            class_names = [names[i] for i in all_class_ids]
                            print("📝 Loaded class names from classes.txt")
                            return class_names
                except Exception as e:
                    print(f"⚠️ Error reading classes.txt: {e}")

            for file in files:
                if file.endswith(('.yaml', '.yml')):
                    try:
                        with open(os.path.join(root, file), 'r') as f:
                            data = yaml.safe_load(f)
                            if 'names' in data:
                                names = data['names']
                                if isinstance(names, list) and len(names) >= max(all_class_ids) + 1:
                                    class_names = [names[i] for i in all_class_ids]
                                    print(f"📝 Loaded class names from {file}")
                                    return class_names
                                elif isinstance(names, dict):
                                    class_names = [names.get(i, f"Class_{i}") for i in all_class_ids]
                                    print(f"📝 Loaded class names from {file}")
                                    return class_names
                    except Exception as e:
                        print(f"⚠️ Error reading {file}: {e}")

        print("📝 Using default class names")
        return class_names

    def select_classes_interactive(self):
        """Interactive class selection with better UX"""
        print(f"\n📋 Available Classes ({len(self.all_classes)} total):")

        for i, class_name in enumerate(self.all_classes):
            print(f"   {i:2d}: {class_name}")

        print(f"\n🎯 Select classes to train:")
        print("   • Type 'all' for all classes")
        print("   • Type numbers separated by commas (e.g., 0,2,5)")
        print("   • Type ranges with dashes (e.g., 0-5,8,10-12)")
        print("   • Press Ctrl+C to cancel")

        while True:
            try:
                selection = input("\n➤ Enter your selection: ").strip()

                if selection.lower() == 'all':
                    selected_indices = list(range(len(self.all_classes)))
                    break

                selected_indices = self.parse_selection(selection)

                if selected_indices:
                    break
                else:
                    print("❌ No valid classes selected. Please try again.")

            except KeyboardInterrupt:
                print("\n🚫 Selection cancelled by user.")
                sys.exit(0)
            except Exception as e:
                print(f"❌ Error: {e}. Please try again.")

        print(f"\n✅ Selected {len(selected_indices)} classes:")
        for i in selected_indices:
            print(f"   {i:2d}: {self.all_classes[i]}")

        return selected_indices

    def parse_selection(self, selection):
        """Parse user selection string"""
        selected_indices = []
        parts = selection.split(',')

        for part in parts:
            part = part.strip()
            if '-' in part:
                try:
                    start, end = map(int, part.split('-'))
                    selected_indices.extend(range(start, end + 1))
                except ValueError:
                    print(f"❌ Invalid range format: {part}")
                    continue
            else:
                try:
                    selected_indices.append(int(part))
                except ValueError:
                    print(f"❌ Invalid number: {part}")
                    continue

        # Remove duplicates and validate
        selected_indices = list(set(selected_indices))
        valid_indices = [i for i in selected_indices if 0 <= i < len(self.all_classes)]

        if len(valid_indices) != len(selected_indices):
            invalid = [i for i in selected_indices if i not in valid_indices]
            print(f"⚠️ Invalid indices ignored: {invalid}")

        return sorted(valid_indices)

    def get_training_parameters(self):
        """Get training parameters from user"""
        print(f"\n⚙️ Training Configuration:")
        print("Press Enter to use default values shown in parentheses")

        params = {}

        try:
            # YOLO Version Selection
            print(f"\n🤖 Select YOLO Version:")
            yolo_versions = {
                0: ("YOLOv8", ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt", "yolov8l.pt", "yolov8x.pt"]),
                1: ("YOLOv9", ["yolov9t.pt", "yolov9s.pt", "yolov9m.pt", "yolov9c.pt", "yolov9e.pt"]),
                2: ("YOLOv10", ["yolov10n.pt", "yolov10s.pt", "yolov10m.pt", "yolov10b.pt", "yolov10l.pt", "yolov10x.pt"]),
                3: ("YOLO11", ["yolo11n.pt", "yolo11s.pt", "yolo11m.pt", "yolo11l.pt", "yolo11x.pt"])
            }

            for i, (version_name, models) in yolo_versions.items():
                print(f"   {i}: {version_name} ({len(models)} models)")

            version_input = input("Select YOLO version (3 for YOLO11): ").strip()
            version_idx = int(version_input) if version_input else 3

            if version_idx not in yolo_versions:
                version_idx = 3
                print("⚠️ Invalid version, using YOLO11")

            selected_version, available_models = yolo_versions[version_idx]
            params['yolo_version'] = selected_version

            print(f"\n📦 Selected: {selected_version}")
            print(f"Available {selected_version} models:")
            for i, model in enumerate(available_models):
                model_size = model.split('.')[0][-1]  # Get the size letter (n, s, m, l, x, etc.)
                size_names = {
                    'n': 'Nano (fastest, least accurate)',
                    't': 'Tiny (very fast, low accuracy)',
                    's': 'Small (fast, good accuracy)',
                    'm': 'Medium (balanced speed/accuracy)',
                    'c': 'Classic (good accuracy)',
                    'b': 'Balanced (optimized)',
                    'l': 'Large (slow, high accuracy)',
                    'x': 'Extra Large (slowest, highest accuracy)',
                    'e': 'Efficient (optimized for deployment)'
                }
                description = size_names.get(model_size, 'Standard model')
                print(f"   {i}: {model} - {description}")

            model_input = input(f"Select model (0 for {available_models[0]}): ").strip()
            model_idx = int(model_input) if model_input else 0
            params['model'] = available_models[model_idx] if 0 <= model_idx < len(available_models) else available_models[0]

            epochs_input = input("\nEpochs (100): ").strip()
            params['epochs'] = int(epochs_input) if epochs_input else 100

            imgsz_input = input("Image size (640): ").strip()
            params['imgsz'] = int(imgsz_input) if imgsz_input else 640

            batch_input = input("Batch size (16): ").strip()
            params['batch'] = int(batch_input) if batch_input else 16

        except ValueError:
            print("⚠️ Invalid input detected. Using default parameters...")
            params = {
                'epochs': 100,
                'imgsz': 640,
                'batch': 16,
                'model': 'yolo11n.pt',
                'yolo_version': 'YOLO11'
            }
        except KeyboardInterrupt:
            print("\n🚫 Configuration cancelled by user.")
            sys.exit(0)

        return params

    def train_model(self, selected_indices, params):
        """Train YOLO model with comprehensive error handling and return model path"""
        try:
            import torch
            from ultralytics import YOLO
        except ImportError as e:
            print(f"❌ Required packages not available: {e}")
            return False, None

        try:
            yolo_version = params.get('yolo_version', 'YOLO11')
            print(f"🚀 Starting {yolo_version} training with {len(selected_indices)} classes...")

            # Create dataset configuration
            class_mapping = self.create_dataset_yaml(selected_indices)

            # Filter labels
            self.filter_labels(class_mapping)

            # Final verification
            if not self.final_dataset_check():
                print("❌ Dataset verification failed")
                return False, None

            # Initialize model
            print(f"🤖 Loading {params['model']} model...")
            model = YOLO(params['model'])

            # Adjust batch size for GPU memory if needed
            if torch.cuda.is_available():
                try:
                    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
                    if gpu_memory < 8 and params['batch'] > 8:
                        params['batch'] = 8
                        print(f"📉 Reduced batch size to {params['batch']} for GPU memory")
                except Exception:
                    pass

            print("🏃 Training started...")
            print(f"Parameters: {yolo_version} {params['model']}, epochs={params['epochs']}, imgsz={params['imgsz']}, batch={params['batch']}")

            yaml_path = os.path.abspath('./dataset.yaml')
            print(f"📄 Using dataset config: {yaml_path}")

            # Create custom run name with YOLO version
            run_name = f"{yolo_version.lower()}_custom"

            # Start training
            results = model.train(
                data=yaml_path,
                epochs=params['epochs'],
                imgsz=params['imgsz'],
                batch=params['batch'],
                device='0' if torch.cuda.is_available() else 'cpu',
                project='./runs/train',
                name=run_name,
                exist_ok=True,
                verbose=True,
                patience=20,
                save_period=max(10, params['epochs'] // 10)
            )

            # Get the trained model path
            model_dir = results.save_dir
            best_model_path = os.path.join(model_dir, 'weights', 'best.pt')
            last_model_path = os.path.join(model_dir, 'weights', 'last.pt')

            # Check which model files exist and prefer 'best.pt'
            if os.path.exists(best_model_path):
                trained_model_path = best_model_path
                model_type = "best"
            elif os.path.exists(last_model_path):
                trained_model_path = last_model_path
                model_type = "last"
            else:
                # Fallback to save_dir if weights folder structure is different
                trained_model_path = str(model_dir)
                model_type = "directory"

            print("🎉 Training completed successfully!")
            print(f"📁 Training results saved to: {model_dir}")
            print(f"🏆 Trained model ({model_type}): {trained_model_path}")

            return True, trained_model_path

        except Exception as e:
            print(f"❌ Training failed: {str(e)}")
            print("\n🔧 Troubleshooting tips:")
            print("• Check dataset paths and file permissions")
            print("• Try reducing batch size or image size")
            print("• Verify all images and labels are valid")
            print("• Check available disk space")
            return False, None

    def create_dataset_yaml(self, selected_indices):
        """Create dataset.yaml configuration"""
        class_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(selected_indices)}
        selected_names = [self.all_classes[i] for i in selected_indices]

        dataset_config = {
            'path': os.path.abspath('./temp_data'),
            'train': 'train/images',
            'val': 'val/images',
            'test': 'test/images',
            'nc': len(selected_indices),
            'names': selected_names
        }

        with open('./dataset.yaml', 'w') as f:
            yaml.dump(dataset_config, f, default_flow_style=False)

        print(f"📄 Created dataset.yaml with {len(selected_indices)} classes")
        return class_mapping

    def filter_labels(self, class_mapping):
        """Filter label files for selected classes"""
        print("🔄 Filtering labels...")

        for root, dirs, files in os.walk('./temp_data'):
            if 'labels' in root:
                for file in files:
                    if file.endswith('.txt'):
                        self.filter_label_file(os.path.join(root, file), class_mapping)

    def filter_label_file(self, label_path, class_mapping):
        """Filter individual label file"""
        try:
            with open(label_path, 'r') as f:
                lines = f.readlines()

            filtered_lines = []
            for line in lines:
                if line.strip():
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        try:
                            class_id = int(parts[0])
                            if class_id in class_mapping:
                                parts[0] = str(class_mapping[class_id])
                                filtered_lines.append(' '.join(parts) + '\n')
                        except ValueError:
                            continue

            with open(label_path, 'w') as f:
                f.writelines(filtered_lines)

        except Exception as e:
            print(f"⚠️ Error filtering {label_path}: {e}")

    def final_dataset_check(self):
        """Final comprehensive check before training"""
        print("🔍 Final dataset verification...")

        # Check required paths
        required_paths = [
            './temp_data/train/images',
            './temp_data/train/labels',
            './temp_data/val/images',
            './temp_data/val/labels',
            './dataset.yaml'
        ]

        for path in required_paths:
            if not os.path.exists(path):
                print(f"❌ Missing: {path}")
                return False

        # Count files and verify content
        train_images = len([f for f in os.listdir('./temp_data/train/images')
                           if f.lower().endswith(tuple(self.dataset_manager.image_extensions))])
        train_labels = len([f for f in os.listdir('./temp_data/train/labels') if f.endswith('.txt')])
        val_images = len([f for f in os.listdir('./temp_data/val/images')
                         if f.lower().endswith(tuple(self.dataset_manager.image_extensions))])
        val_labels = len([f for f in os.listdir('./temp_data/val/labels') if f.endswith('.txt')])

        print(f"📊 Dataset summary:")
        print(f"   🏋️ Training: {train_images} images, {train_labels} labels")
        print(f"   ✅ Validation: {val_images} images, {val_labels} labels")

        if train_images == 0:
            print("❌ No training images found")
            return False

        if val_images == 0:
            print("❌ No validation images found")
            return False

        # Verify dataset.yaml content
        try:
            with open('./dataset.yaml', 'r') as f:
                config = yaml.safe_load(f)
                required_keys = ['path', 'train', 'val', 'nc', 'names']
                for key in required_keys:
                    if key not in config:
                        print(f"❌ Missing key in dataset.yaml: {key}")
                        return False
        except Exception as e:
            print(f"❌ Error reading dataset.yaml: {e}")
            return False

        print("✅ Final verification passed - ready for training!")
        return True

    def run_interactive(self):
        """Run interactive CLI training with improved file handling"""
        print("\n🚀 YOLO11 Training Tool (Interactive Mode)")
        print("=" * 50)

        # Get ZIP file with better error handling
        zip_path = self.get_zip_file_interactive()
        if not zip_path:
            print("❌ Unable to load ZIP file. Exiting.")
            return

        # Load data
        if not self.load_data(zip_path):
            print("❌ Failed to load data. Exiting.")
            return

        # Select classes
        selected_indices = self.select_classes_interactive()

        # Get training parameters
        params = self.get_training_parameters()

        # Show summary
        print(f"\n📋 Training Summary:")
        print(f"   📊 Dataset: {os.path.basename(zip_path)}")
        print(f"   🎯 Classes: {len(selected_indices)} selected")
        print(f"   🤖 YOLO Version: {params.get('yolo_version', 'YOLO11')}")
        print(f"   📦 Model: {params['model']}")
        print(f"   📈 Epochs: {params['epochs']}")
        print(f"   🖼️ Image size: {params['imgsz']}")
        print(f"   📦 Batch size: {params['batch']}")

        # Confirm training
        print(f"\n⚡ Ready to start training!")
        try:
            confirm = input("Continue? (y/N): ").strip().lower()
            if confirm in ['y', 'yes']:
                success, model_path = self.train_model(selected_indices, params)
                if success and model_path:
                    print("\n🎉 Training completed successfully!")
                    print(f"📁 Check './runs/train/{params.get('yolo_version', 'yolo11').lower()}_custom' for all results")
                    print(f"🏆 Trained model: {model_path}")
                    return model_path
                else:
                    print("\n❌ Training failed.")
                    return None
            else:
                print("🚫 Training cancelled.")
                return None
        except KeyboardInterrupt:
            print("\n🚫 Training cancelled by user.")
            return None


def train_yolo_simple(zip_path=None, classes="all", epochs=100, imgsz=640, batch=16, model="yolo11n.pt", yolo_version="auto"):
    """
    Simple function for Jupyter/Colab environments with robust file handling and YOLO version selection

    Parameters:
    - zip_path: Path to the training data ZIP file
    - classes: "all" or list of class indices like "0,1,2" or [0,1,2]
    - epochs: Number of training epochs (default: 100)
    - imgsz: Image size for training (default: 640)
    - batch: Batch size (default: 16)
    - model: Model name (default: "yolo11n.pt")
    - yolo_version: YOLO version - "auto", "yolov8", "yolov9", "yolov10", "yolo11" (default: "auto")

    Returns:
    - Path to the trained model if successful, None if failed
    """

    print("🚀 YOLO Simple Training")
    print("=" * 30)

    # Check packages first
    if not check_and_install_packages():
        print("❌ Failed to install required packages")
        return None

    cli = YOLO11TrainerCLI()

    if zip_path is None:
        print("Interactive mode - please provide input when prompted")
        return cli.run_interactive()

    # Normalize path
    zip_path = FileHandler.normalize_path(zip_path)

    # Validate ZIP file
    is_valid, message = FileHandler.is_valid_zip_detailed(zip_path)
    if not is_valid:
        print(f"❌ {message}")
        return None

    print(f"✅ {message}")

    # Load data
    if not cli.load_data(zip_path):
        print("❌ Failed to load data")
        return None

    # Parse classes
    if isinstance(classes, str):
        if classes.lower() == "all":
            selected_indices = list(range(len(cli.all_classes)))
        else:
            try:
                selected_indices = [int(x.strip()) for x in classes.split(',')]
            except ValueError:
                print("❌ Invalid class format. Use 'all' or '0,1,2'")
                return None
    elif isinstance(classes, list):
        selected_indices = classes
    else:
        print("❌ Classes must be 'all', '0,1,2', or [0,1,2]")
        return None

    # Validate indices
    valid_indices = [i for i in selected_indices if 0 <= i < len(cli.all_classes)]
    if len(valid_indices) != len(selected_indices):
        print("⚠️ Some class indices were invalid and ignored")

    if not valid_indices:
        print("❌ No valid class indices provided")
        return None

    print(f"✅ Selected classes: {[cli.all_classes[i] for i in valid_indices]}")

    # Handle YOLO version and model selection
    yolo_models = {
        "yolov8": ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt", "yolov8l.pt", "yolov8x.pt"],
        "yolov9": ["yolov9t.pt", "yolov9s.pt", "yolov9m.pt", "yolov9c.pt", "yolov9e.pt"],
        "yolov10": ["yolov10n.pt", "yolov10s.pt", "yolov10m.pt", "yolov10b.pt", "yolov10l.pt", "yolov10x.pt"],
        "yolo11": ["yolo11n.pt", "yolo11s.pt", "yolo11m.pt", "yolo11l.pt", "yolo11x.pt"]
    }

    # Auto-detect YOLO version from model name if version is "auto"
    detected_version = yolo_version
    if yolo_version.lower() == "auto":
        for version, models in yolo_models.items():
            if model in models:
                detected_version = version
                break
        else:
            detected_version = "yolo11"  # Default fallback
            print(f"⚠️ Could not detect YOLO version from model '{model}', using YOLO11")

    # Validate model exists for the version
    if detected_version.lower() in yolo_models:
        available_models = yolo_models[detected_version.lower()]
        if model not in available_models:
            print(f"⚠️ Model '{model}' not available for {detected_version.upper()}")
            print(f"Available models: {available_models}")
            model = available_models[0]  # Use first model as fallback
            print(f"Using fallback model: {model}")

    # Train model
    params = {
        'epochs': epochs,
        'imgsz': imgsz,
        'batch': batch,
        'model': model,
        'yolo_version': detected_version.upper()
    }

    print(f"\n🚀 Starting {detected_version.upper()} training with {model}")
    print(f"📊 Parameters: epochs={epochs}, imgsz={imgsz}, batch={batch}")

    success, model_path = cli.train_model(valid_indices, params)

    if success and model_path:
        print("\n🎉 Training completed successfully!")
        print(f"📁 Check './runs/train/{detected_version.lower()}_custom' for all results")
        print(f"🏆 Trained model: {model_path}")
        return model_path
    else:
        print("\n❌ Training failed. Check error messages above.")
        return None


def is_jupyter_environment():
    """Check if running in Jupyter/Colab environment"""
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except ImportError:
        return False


def main():
    """Main entry point with improved error handling"""
    print("🚀 YOLO Training Tool (Multi-Version) - Initializing...")

    # Check and install packages first
    if not check_and_install_packages():
        print("❌ Failed to set up required packages. Exiting.")
        return

    # Check environment
    if is_jupyter_environment():
        print("🔬 Detected Jupyter/Colab environment")
        print("💡 Use train_yolo_simple() function for easy training:")
        print("   model_path = train_yolo_simple('/path/to/data.zip', classes='all', epochs=100)")
        print("   model_path = train_yolo_simple('/path/to/data.zip', model='yolov8n.pt')")
        print("\n📋 Starting interactive CLI mode...")
        cli = YOLO11TrainerCLI()
        return cli.run_interactive()

    # Parse command line arguments
    parser = argparse.ArgumentParser(
        description='YOLO Training Tool - Multi-version support with fixed dataset and file handling',
        epilog='''
Examples:
  python yolo11_trainer.py --cli
  python yolo11_trainer.py --zip data.zip --classes all --epochs 50
  python yolo11_trainer.py --zip data.zip --classes 0,2,5 --epochs 100 --model yolov8n.pt

Supported YOLO versions:
  • YOLOv8: yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
  • YOLOv9: yolov9t.pt, yolov9s.pt, yolov9m.pt, yolov9c.pt, yolov9e.pt
  • YOLOv10: yolov10n.pt, yolov10s.pt, yolov10m.pt, yolov10b.pt, yolov10l.pt, yolov10x.pt
  • YOLO11: yolo11n.pt, yolo11s.pt, yolo11m.pt, yolo11l.pt, yolo11x.pt
        ''',
        formatter_class=argparse.RawDescriptionHelpFormatter
    )

    parser.add_argument('--gui', action='store_true', help='Force GUI mode')
    parser.add_argument('--cli', action='store_true', help='Force CLI mode')
    parser.add_argument('--zip', type=str, help='Path to training data ZIP file')
    parser.add_argument('--classes', type=str, help='Class indices ("all" or "0,1,2")')
    parser.add_argument('--epochs', type=int, default=100, help='Training epochs')
    parser.add_argument('--imgsz', type=int, default=640, help='Image size')
    parser.add_argument('--batch', type=int, default=16, help='Batch size')
    parser.add_argument('--model', type=str, default='yolo11n.pt', help='YOLO model')

    try:
        args = parser.parse_args()
    except SystemExit:
        print("Starting interactive CLI mode...")
        cli = YOLO11TrainerCLI()
        cli.run_interactive()
        return

    # Determine interface mode
    if args.cli or (not GUI_AVAILABLE and not args.gui):
        # CLI mode
        cli = YOLO11TrainerCLI()

        if args.zip and args.classes:
            # Non-interactive mode
            # Normalize ZIP path
            zip_path = FileHandler.normalize_path(args.zip)

            # Validate ZIP file
            is_valid, message = FileHandler.is_valid_zip_detailed(zip_path)
            if not is_valid:
                print(f"❌ {message}")
                return

            if cli.load_data(zip_path):
                if args.classes.lower() == 'all':
                    selected_indices = list(range(len(cli.all_classes)))
                else:
                    selected_indices = cli.parse_selection(args.classes)

                if selected_indices:
                    params = {
                        'epochs': args.epochs,
                        'imgsz': args.imgsz,
                        'batch': args.batch,
                        'model': args.model,
                        'yolo_version': 'YOLO11'  # Default for CLI
                    }
                    success, model_path = cli.train_model(selected_indices, params)
                    if success and model_path:
                        print(f"\n🏆 Training completed! Model saved at: {model_path}")
                    else:
                        print("\n❌ Training failed.")
                else:
                    print("❌ No valid class indices provided")
        else:
            # Interactive mode
            cli.run_interactive()

    elif args.gui or GUI_AVAILABLE:
        # GUI mode (would need to be implemented with the new FileHandler and YOLO version support)
        if not GUI_AVAILABLE:
            print("❌ GUI not available. Use --cli flag.")
            return

        print("🚀 GUI mode would be launched here with multi-YOLO support...")
        print("💡 For now, using CLI mode...")
        cli = YOLO11TrainerCLI()
        return cli.run_interactive()

    else:
        print("❌ No interface available. Use --cli flag.")


if __name__ == "__main__":
    main()
else:
    # When imported as module
    if is_jupyter_environment():
        print("🔬 YOLO Training Tool loaded in Jupyter/Colab")
        print("💡 Quick start:")
        print("   train_yolo_simple('/path/to/data.zip', classes='all', epochs=100)")
        print("   train_yolo_simple('/path/to/data.zip', classes='0,2,5', epochs=50)")
        print("   train_yolo_simple('/path/to/data.zip', model='yolov8n.pt', yolo_version='yolov8')")
        print("   train_yolo_simple('/path/to/data.zip', model='yolov10m.pt')  # Auto-detects version")

YOLO11 Training Tool
GUI not available: no display name and no $DISPLAY environment variable
Running in command-line mode...

To enable GUI:
- On Linux/WSL: Install X server (Xming, VcXsrv, or X11)
- On SSH: Use 'ssh -X' for X11 forwarding
- On headless servers: Use CLI mode with --cli flag
🚀 YOLO Training Tool (Multi-Version) - Initializing...
🔍 Checking required packages...
✅ PyTorch 2.6.0+cu124 is available
✅ CUDA available with 1 GPU(s)
   GPU 0: Tesla T4
❌ Ultralytics not found

📦 Installing 1 missing package(s)...

⏳ Installing ultralytics...
✅ ultralytics installed successfully

🔄 Reloading modules...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ All packages loaded successfully
🔬 Detected Jupyter/Colab environment
💡

🏃 Training started...
Parameters: YOLO11 yolo11m.pt, epochs=400, imgsz=640, batch=16
📄 Using dataset config: /content/dataset.yaml
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=400, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11_custom, nbs=64, nms=False

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

 13                  -1  1   1642496  ultralytics.nn.modules.block.C3k2            [1024, 512, 1, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    542720  ultralytics.nn.modules.block.C3k2            [1024, 256, 1, True]          
 17                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 19                  -1  1   1511424  ultralytics.nn.modules.block.C3k2            [768, 512, 1, True]           
 20                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
 21            [-1, 10]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

ENHANCED SEGFORMER FINE-TUNING SYSTEM

In [ ]:
# ===============================================================================
# COMPLETE SEGMENTATION PROCESS - FULLY EXPLAINED WITH PERFORMANCE TUNING
# ===============================================================================
# Every command explained + Performance optimization annotations
# Learn exactly how segmentation works and how to improve it
# ===============================================================================

import warnings
warnings.filterwarnings('ignore')  # Hide unnecessary warnings

# Core libraries with explanations
import torch  # PyTorch: Main deep learning framework
import torch.nn as nn  # Neural network components (layers, loss functions)
import torch.nn.functional as F  # Functional operations (interpolation, etc.)
from torch.utils.data import Dataset, DataLoader  # Data handling for training
from transformers import (
    SegformerImageProcessor,  # Preprocesses images for Segformer
    SegformerForSemanticSegmentation,  # The segmentation model itself
    TrainingArguments,  # Training configuration
    Trainer  # Handles training loop
)
import numpy as np  # Numerical array operations
from PIL import Image  # Image loading and basic operations
import os  # File system operations
import cv2  # Computer vision operations (boundaries, contours)
from tqdm import tqdm  # Progress bars
import matplotlib.pyplot as plt  # Plotting and visualization
import json  # JSON file handling
from datetime import datetime  # Timestamps
import time  # Performance timing
from sklearn.metrics import accuracy_score, jaccard_score  # Evaluation metrics
import albumentations as A  # Data augmentation library

# Check environment
try:
    from google.colab import files
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# ===============================================================================
# 🎯 SEGMENTATION PROCESS EXPLAINED - COMPLETE PIPELINE
# ===============================================================================
"""
📚 HOW SEMANTIC SEGMENTATION WORKS - COMPLETE EXPLANATION:

🔍 INPUT PROCESSING:
1. Take RGB image (Height × Width × 3)
2. Resize to model input size (typically 512×512 or 1024×1024)
3. Normalize pixel values (0-255 → 0-1 range)
4. Convert to tensor format for neural network

🧠 NEURAL NETWORK PROCESSING:
1. Encoder: Extract features at multiple scales
   - Early layers: detect edges, textures
   - Middle layers: detect shapes, objects
   - Deep layers: detect complex patterns
2. Decoder: Combine features to make pixel predictions
   - Upsampling: restore original image size
   - Feature fusion: combine different scale information
3. Output: Probability map (Height × Width × NumClasses)

🎯 PREDICTION GENERATION:
1. For each pixel, get probability for each class
2. Take argmax (highest probability) = final class
3. Create segmentation map (Height × Width)

🎨 VISUALIZATION:
1. Map class IDs to colors
2. Create colored mask
3. Add boundaries for clarity
4. Blend with original image

⚡ PERFORMANCE FACTORS:
- Model architecture (larger = more accurate but slower)
- Input resolution (higher = more detail but slower)
- Preprocessing quality
- Post-processing techniques
"""

class CompleteSegmentationExplained:
    """
    🔬 COMPLETE SEGMENTATION SYSTEM WITH DETAILED EXPLANATIONS

    This class demonstrates every step of the segmentation process
    with detailed explanations and performance tuning options.
    """

    def __init__(self):
        """
        🏗️ SYSTEM INITIALIZATION - DETAILED SETUP

        Sets up the complete segmentation pipeline with all components
        needed for high-performance semantic segmentation.
        """
        print("🚀 INITIALIZING COMPLETE SEGMENTATION SYSTEM")
        print("=" * 70)

        # 🔧 PERFORMANCE TUNING PARAMETER #1: Device Selection
        # GPU vs CPU dramatically affects speed (10-100x difference)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"💻 Computing device: {self.device}")

        if torch.cuda.is_available():
            # Display GPU information for performance reference
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"🔥 GPU: {gpu_name}")
            print(f"💾 GPU Memory: {gpu_memory:.1f} GB")

            # 🔧 PERFORMANCE TUNING PARAMETER #2: Memory Optimization
            # Enable memory efficient attention if available
            torch.backends.cuda.enable_flash_sdp(True)
        else:
            print("⚠️  Using CPU - expect slower performance")

        # Initialize model components
        self.pretrained_model = None
        self.processor = None
        self.model_info = {}

        # 🔧 PERFORMANCE TUNING PARAMETER #3: Processing Settings
        # These settings significantly affect quality vs speed trade-off
        self.processing_config = {
            'input_size': 512,          # 🎯 CRITICAL: Higher = better quality, slower speed
            'batch_size': 1,            # 🎯 CRITICAL: Higher = faster, more memory usage
            'interpolation_mode': 'bilinear',  # 🎯 CRITICAL: bilinear vs nearest
            'align_corners': False,     # 🎯 CRITICAL: Affects upsampling quality
            'confidence_threshold': 0.5,  # 🎯 CRITICAL: Filter low-confidence predictions
            'boundary_thickness': 2,    # Visual: Boundary line thickness
            'blend_alpha': 0.6         # Visual: Original image opacity in overlay
        }

        # 🔧 PERFORMANCE TUNING PARAMETER #4: Color Mapping
        # High-contrast colors improve visual clarity
        self.performance_colors = self._create_performance_colors()

        # Setup directories
        self.setup_directories()

        print("✅ System initialized with performance optimizations!")
        self._print_performance_tips()

    def _print_performance_tips(self):
        """Print performance optimization tips"""
        print("\n🚀 PERFORMANCE OPTIMIZATION TIPS:")
        print("  🎯 For SPEED: Reduce input_size to 256-512")
        print("  🎯 For QUALITY: Increase input_size to 1024+")
        print("  🎯 For MEMORY: Reduce batch_size")
        print("  🎯 For GPU: Enable mixed precision (fp16)")
        print("  🎯 For CPU: Use smaller models (segformer-b0)")

    def setup_directories(self):
        """Create organized directory structure"""
        self.dirs = {
            'models': './segmentation_models',
            'outputs': './video_outputs',
            'results': './image_results',
            'temp': './temp_processing'
        }

        for path in self.dirs.values():
            os.makedirs(path, exist_ok=True)

# ===============================================================================
# 🔧 PERFORMANCE-CRITICAL MODEL LOADING
# ===============================================================================
    def load_model_with_performance_optimization(self, model_path=None):
        """
        📤 LOAD MODEL WITH PERFORMANCE OPTIMIZATION

        🔧 PERFORMANCE ANNOTATIONS:
        - Model loading affects all subsequent processing speed
        - Larger models (b1-b5) = better accuracy, slower inference
        - Smaller models (b0) = faster inference, lower accuracy
        - Memory optimization critical for large models
        """
        print("\n📤 LOADING MODEL WITH PERFORMANCE OPTIMIZATION")
        print("=" * 60)

        # Get model path
        if model_path is None:
            if COLAB_ENV:
                uploaded = files.upload()
                if not uploaded:
                    return False
                model_path = list(uploaded.keys())[0]
            else:
                model_path = input("📁 Enter model path: ").strip()

        try:
            # 🔧 PERFORMANCE TUNING: Model Architecture Selection
            print("🔧 PERFORMANCE OPTIMIZATION: Selecting base architecture...")

            # Different base models with performance characteristics
            base_models = {
                'b0': "nvidia/segformer-b0-finetuned-cityscapes-1024-1024",  # Fastest
                'b1': "nvidia/segformer-b1-finetuned-cityscapes-1024-1024",  # Balanced
                'b2': "nvidia/segformer-b2-finetuned-cityscapes-1024-1024",  # Better quality
                'b3': "nvidia/segformer-b3-finetuned-cityscapes-1024-1024",  # High quality
                'b4': "nvidia/segformer-b4-finetuned-cityscapes-1024-1024",  # Very high quality
                'b5': "nvidia/segformer-b5-finetuned-cityscapes-1024-1024"   # Best quality, slowest
            }

            print("📊 Available architectures (performance vs quality):")
            print("  🏃 b0: Fastest (4x speed, 85% accuracy)")
            print("  ⚖️  b1: Balanced (3x speed, 87% accuracy)")
            print("  🎯 b2: Good (2x speed, 89% accuracy)")
            print("  🔥 b3: Better (1.5x speed, 91% accuracy)")
            print("  🌟 b4: Excellent (1x speed, 93% accuracy)")
            print("  🏆 b5: Best (0.7x speed, 95% accuracy)")

            # 🔧 PERFORMANCE CHOICE: Let user select or auto-detect
            architecture = 'b0'  # Default to fastest for demonstration
            base_model_name = base_models[architecture]
            print(f"🚀 Using {architecture} architecture for optimal performance")

            # Load checkpoint if provided
            if model_path.endswith(('.pth', '.pt')):
                checkpoint = torch.load(model_path, map_location='cpu')

                # Extract state dict
                if 'model_state_dict' in checkpoint:
                    state_dict = checkpoint['model_state_dict']
                elif 'state_dict' in checkpoint:
                    state_dict = checkpoint['state_dict']
                else:
                    state_dict = checkpoint

                # Detect classes
                num_classes = self._detect_classes_optimized(state_dict)
                print(f"🎯 Detected {num_classes} classes")

                # 🔧 PERFORMANCE OPTIMIZATION: Load with memory efficiency
                print("🔧 Loading model with memory optimization...")
                self.pretrained_model = SegformerForSemanticSegmentation.from_pretrained(
                    base_model_name,
                    num_labels=num_classes,
                    ignore_mismatched_sizes=True,
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32  # 🔧 CRITICAL: Half precision for speed
                )

                # Load weights
                missing, unexpected = self.pretrained_model.load_state_dict(state_dict, strict=False)
                print(f"✅ Weights loaded: {len(state_dict)-len(missing)}/{len(state_dict)} parameters")

            else:
                # Load HuggingFace model
                self.pretrained_model = SegformerForSemanticSegmentation.from_pretrained(model_path)
                num_classes = self.pretrained_model.config.num_labels

            # 🔧 PERFORMANCE OPTIMIZATION: Setup processor with optimal settings
            self.processor = SegformerImageProcessor.from_pretrained(
                base_model_name,
                size=self.processing_config['input_size'],  # 🔧 CRITICAL: Controls quality vs speed
                do_resize=True,
                do_normalize=True
            )

            # 🔧 PERFORMANCE OPTIMIZATION: Model optimizations
            self.pretrained_model.to(self.device)
            self.pretrained_model.eval()  # Set to evaluation mode

            # Enable optimizations
            if torch.cuda.is_available():
                # 🔧 PERFORMANCE BOOST: Compile model for faster inference (PyTorch 2.0+)
                try:
                    self.pretrained_model = torch.compile(self.pretrained_model, mode='max-autotune')
                    print("🚀 Model compiled for maximum performance")
                except:
                    print("⚠️  Model compilation not available")

            # Store model info
            self.model_info = {
                'num_classes': num_classes,
                'architecture': architecture,
                'input_size': self.processing_config['input_size'],
                'device': str(self.device)
            }

            print("✅ Model loaded with performance optimizations!")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            return False

    def _detect_classes_optimized(self, state_dict):
        """Optimized class detection"""
        for key, tensor in state_dict.items():
            if any(x in key.lower() for x in ['classifier', 'decode_head', 'head']):
                if 'weight' in key and len(tensor.shape) >= 2:
                    return tensor.shape[0]
        return 19  # Default cityscapes classes

# ===============================================================================
# 🎯 CORE SEGMENTATION PROCESS - STEP BY STEP EXPLANATION
# ===============================================================================
    def process_image_complete_explanation(self, image_path, save_results=True):
        """
        🖼️ COMPLETE IMAGE SEGMENTATION PROCESS - EVERY STEP EXPLAINED

        This function demonstrates the COMPLETE segmentation pipeline:
        1. Image Loading & Preprocessing
        2. Neural Network Inference
        3. Post-processing & Optimization
        4. Visualization & Analysis
        5. Performance Measurement

        🔧 PERFORMANCE CRITICAL SECTIONS ARE MARKED WITH ANNOTATIONS
        """
        print(f"\n🖼️ COMPLETE SEGMENTATION PROCESS - STEP BY STEP")
        print("=" * 70)
        print(f"📁 Processing: {os.path.basename(image_path)}")

        if not self.pretrained_model:
            print("❌ No model loaded! Load a model first.")
            return None

        start_total = time.time()

        try:
            # ================================================================
            # STEP 1: IMAGE LOADING & PREPROCESSING
            # ================================================================
            print("\n1️⃣ IMAGE LOADING & PREPROCESSING")
            print("   🔍 This step prepares the image for the neural network")
            print("   📊 Performance impact: ~5% of total time")

            step1_start = time.time()

            # Load image
            print("   📁 Loading image file...")
            original_image = Image.open(image_path).convert("RGB")
            original_np = np.array(original_image)
            print(f"      ✅ Loaded: {original_image.size} pixels")

            # 🔧 PERFORMANCE CRITICAL: Image preprocessing
            print("   ⚙️  Preprocessing for neural network...")
            print("      🔧 PERFORMANCE FACTOR: Input size affects quality vs speed")
            print(f"      📏 Target size: {self.processing_config['input_size']}×{self.processing_config['input_size']}")

            # Preprocess with optimized settings
            inputs = self.processor(
                original_image,
                return_tensors="pt",
                do_resize=True,
                size=self.processing_config['input_size']  # 🔧 CRITICAL: Size vs quality trade-off
            )

            # Move to device (GPU/CPU)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            step1_time = time.time() - step1_start
            print(f"   ⏱️  Preprocessing completed: {step1_time:.3f}s")

            # ================================================================
            # STEP 2: NEURAL NETWORK INFERENCE
            # ================================================================
            print("\n2️⃣ NEURAL NETWORK INFERENCE")
            print("   🔍 The neural network analyzes every pixel")
            print("   📊 Performance impact: ~80% of total time")
            print("   🧠 Process: Input → Encoder → Decoder → Predictions")

            step2_start = time.time()

            # 🔧 PERFORMANCE CRITICAL: Neural network inference
            print("   🧠 Running neural network inference...")
            print("      🔧 PERFORMANCE FACTORS:")
            print("         • Model size (b0-b5)")
            print("         • Input resolution")
            print("         • Device (GPU vs CPU)")
            print("         • Batch size")
            print("         • Mixed precision")

            with torch.no_grad():  # 🔧 CRITICAL: Disable gradients for inference
                # Enable autocast for mixed precision (speed boost)
                if torch.cuda.is_available():
                    with torch.autocast(device_type='cuda', dtype=torch.float16):
                        outputs = self.pretrained_model(**inputs)
                else:
                    outputs = self.pretrained_model(**inputs)

            step2_time = time.time() - step2_start
            print(f"   ⏱️  Inference completed: {step2_time:.3f}s")
            print(f"   📊 Output shape: {outputs.logits.shape}")

            # ================================================================
            # STEP 3: PREDICTION POST-PROCESSING
            # ================================================================
            print("\n3️⃣ PREDICTION POST-PROCESSING")
            print("   🔍 Convert neural network output to final segmentation")
            print("   📊 Performance impact: ~10% of total time")

            step3_start = time.time()

            # 🔧 PERFORMANCE CRITICAL: Upsampling to original size
            print("   📏 Upsampling predictions to original image size...")
            print("      🔧 PERFORMANCE FACTORS:")
            print("         • Interpolation method (bilinear vs nearest)")
            print("         • Align corners setting")
            print("         • Output resolution")

            predictions = F.interpolate(
                outputs.logits,
                size=original_image.size[::-1],  # (height, width)
                mode=self.processing_config['interpolation_mode'],  # 🔧 CRITICAL: Quality vs speed
                align_corners=self.processing_config['align_corners']  # 🔧 CRITICAL: Alignment quality
            )

            # Convert to class map
            print("   🎯 Converting probabilities to class predictions...")
            print("      🔧 Process: For each pixel, select class with highest probability")

            # Get raw probabilities for analysis
            probabilities = F.softmax(predictions, dim=1)
            confidence_map = torch.max(probabilities, dim=1)[0].cpu().numpy()

            # Get final predictions
            predicted_map = predictions.squeeze().cpu().numpy().argmax(axis=0)

            # 🔧 PERFORMANCE OPTIMIZATION: Confidence filtering
            if self.processing_config['confidence_threshold'] > 0:
                print(f"   🔧 Applying confidence threshold: {self.processing_config['confidence_threshold']}")
                low_confidence = confidence_map < self.processing_config['confidence_threshold']
                predicted_map[low_confidence] = 0  # Set low confidence to background

            step3_time = time.time() - step3_start
            print(f"   ⏱️  Post-processing completed: {step3_time:.3f}s")

            # ================================================================
            # STEP 4: RESULT ANALYSIS
            # ================================================================
            print("\n4️⃣ SEGMENTATION RESULT ANALYSIS")
            print("   🔍 Analyzing what the model detected")

            # Analyze detected classes
            unique_classes = np.unique(predicted_map)
            class_stats = {}

            print(f"   📊 Classes detected: {len(unique_classes)}")
            total_pixels = predicted_map.size

            for class_id in unique_classes:
                count = np.sum(predicted_map == class_id)
                percentage = (count / total_pixels) * 100
                avg_confidence = np.mean(confidence_map[predicted_map == class_id])

                class_stats[class_id] = {
                    'pixels': count,
                    'percentage': percentage,
                    'confidence': avg_confidence
                }

                class_name = f"Class_{class_id}"  # You can customize this
                print(f"      🎯 {class_name}: {percentage:.1f}% (conf: {avg_confidence:.2f})")

            # ================================================================
            # STEP 5: VISUALIZATION CREATION
            # ================================================================
            print("\n5️⃣ CREATING ENHANCED VISUALIZATIONS")
            print("   🔍 Creating clear, professional visualizations")
            print("   📊 Performance impact: ~5% of total time")

            step5_start = time.time()

            # Create visualizations
            visualizations = self._create_enhanced_visualizations_explained(
                original_np, predicted_map, confidence_map, unique_classes
            )

            step5_time = time.time() - step5_start
            print(f"   ⏱️  Visualization creation: {step5_time:.3f}s")

            # ================================================================
            # STEP 6: PERFORMANCE SUMMARY
            # ================================================================
            total_time = time.time() - start_total

            print(f"\n6️⃣ PERFORMANCE SUMMARY")
            print("=" * 40)
            print(f"   ⏱️  Total processing time: {total_time:.3f}s")
            print(f"   📊 Time breakdown:")
            print(f"      • Preprocessing: {step1_time:.3f}s ({step1_time/total_time*100:.1f}%)")
            print(f"      • Neural network: {step2_time:.3f}s ({step2_time/total_time*100:.1f}%)")
            print(f"      • Post-processing: {step3_time:.3f}s ({step3_time/total_time*100:.1f}%)")
            print(f"      • Visualization: {step5_time:.3f}s ({step5_time/total_time*100:.1f}%)")

            # Calculate throughput
            throughput = (original_image.size[0] * original_image.size[1]) / total_time
            print(f"   🚀 Throughput: {throughput/1000000:.1f} megapixels/second")

            # ================================================================
            # STEP 7: DISPLAY AND SAVE RESULTS
            # ================================================================
            if save_results:
                print(f"\n7️⃣ SAVING RESULTS")
                save_dir = self._save_complete_results(
                    image_path, original_np, visualizations, class_stats, total_time
                )
                print(f"   💾 Results saved to: {save_dir}")

            # Display results
            self._display_complete_results(original_np, visualizations)

            print(f"\n🎉 COMPLETE SEGMENTATION FINISHED!")
            print(f"✅ Successfully processed {os.path.basename(image_path)}")

            return {
                'original': original_np,
                'prediction_map': predicted_map,
                'confidence_map': confidence_map,
                'visualizations': visualizations,
                'class_stats': class_stats,
                'performance': {
                    'total_time': total_time,
                    'inference_time': step2_time,
                    'throughput_mpps': throughput/1000000
                }
            }

        except Exception as e:
            print(f"❌ Error during processing: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

# ===============================================================================
# 🎨 ADVANCED VISUALIZATION WITH PERFORMANCE OPTIMIZATION
# ===============================================================================
    def _create_enhanced_visualizations_explained(self, original, prediction_map, confidence_map, unique_classes):
        """
        🎨 CREATE ENHANCED VISUALIZATIONS - COMPLETE PROCESS

        🔧 PERFORMANCE ANNOTATIONS:
        - Visualization quality vs processing speed trade-offs
        - Memory usage optimization for large images
        - Efficient boundary detection algorithms
        """
        print("   🎨 Creating enhanced visualizations...")

        h, w = prediction_map.shape

        # 🔧 PERFORMANCE OPTIMIZATION: Efficient color mapping
        print("      🔧 PERFORMANCE: Using optimized color mapping...")
        colors = self.performance_colors

        # Create base colored mask
        colored_mask = np.zeros((h, w, 3), dtype=np.uint8)
        for class_id in unique_classes:
            if class_id < len(colors):
                mask = prediction_map == class_id
                colored_mask[mask] = colors[class_id]

        # 🔧 PERFORMANCE CRITICAL: Boundary detection
        print("      🔧 PERFORMANCE: Efficient boundary detection...")
        boundary_mask = self._create_optimized_boundaries(prediction_map)

        # Create enhanced mask with boundaries
        enhanced_mask = colored_mask.copy()
        enhanced_mask[boundary_mask > 0] = [255, 255, 255]  # White boundaries

        # 🔧 PERFORMANCE OPTIMIZATION: Confidence visualization
        print("      🔧 PERFORMANCE: Creating confidence overlay...")
        confidence_overlay = self._create_confidence_overlay(original, confidence_map)

        # Create different blend modes
        blend_standard = cv2.addWeighted(
            original,
            self.processing_config['blend_alpha'],
            colored_mask,
            1 - self.processing_config['blend_alpha'],
            0
        )

        blend_enhanced = cv2.addWeighted(
            original,
            self.processing_config['blend_alpha'],
            enhanced_mask,
            1 - self.processing_config['blend_alpha'],
            0
        )

        return {
            'colored_mask': colored_mask,
            'enhanced_mask': enhanced_mask,
            'boundary_mask': boundary_mask,
            'confidence_overlay': confidence_overlay,
            'blend_standard': blend_standard,
            'blend_enhanced': blend_enhanced
        }

    def _create_optimized_boundaries(self, prediction_map):
        """
        🔧 OPTIMIZED BOUNDARY DETECTION

        Uses efficient algorithms for boundary detection:
        - Morphological gradient for speed
        - Sobel edge detection for quality
        - Optimized kernel operations
        """
        # 🔧 PERFORMANCE: Use morphological operations (faster than Sobel)
        kernel = cv2.getStructuringElement(
            cv2.MORPH_RECT,
            (self.processing_config['boundary_thickness'], self.processing_config['boundary_thickness'])
        )

        # Create boundaries
        boundaries = cv2.morphologyEx(
            prediction_map.astype(np.uint8),
            cv2.MORPH_GRADIENT,
            kernel
        )

        return (boundaries > 0).astype(np.uint8) * 255

    def _create_confidence_overlay(self, original, confidence_map):
        """Create confidence-based overlay"""
        # Normalize confidence to 0-255
        conf_norm = (confidence_map * 255).astype(np.uint8)

        # Create color map (red = low confidence, green = high confidence)
        confidence_colored = cv2.applyColorMap(conf_norm, cv2.COLORMAP_RdYlGn)
        confidence_colored = cv2.cvtColor(confidence_colored, cv2.COLOR_BGR2RGB)

        # Blend with original
        confidence_overlay = cv2.addWeighted(original, 0.7, confidence_colored, 0.3, 0)

        return confidence_overlay

    def _create_performance_colors(self):
        """
        🎨 CREATE HIGH-PERFORMANCE COLOR MAPPING

        🔧 PERFORMANCE CONSIDERATIONS:
        - High contrast colors for visibility
        - Distinct hues for easy differentiation
        - Optimized color space for display
        """
        return [
            [64, 64, 64],       # Background - Dark gray
            [255, 0, 0],        # Class 1 - Bright red
            [0, 255, 0],        # Class 2 - Bright green
            [0, 0, 255],        # Class 3 - Bright blue
            [255, 255, 0],      # Class 4 - Yellow
            [255, 0, 255],      # Class 5 - Magenta
            [0, 255, 255],      # Class 6 - Cyan
            [255, 128, 0],      # Class 7 - Orange
            [128, 0, 255],      # Class 8 - Purple
            [255, 192, 203],    # Class 9 - Pink
            [50, 205, 50],      # Class 10 - Lime green
            [255, 69, 0],       # Class 11 - Red orange
            [138, 43, 226],     # Class 12 - Blue violet
            [255, 20, 147],     # Class 13 - Deep pink
            [0, 191, 255],      # Class 14 - Deep sky blue
            [255, 215, 0],      # Class 15 - Gold
            [220, 20, 60],      # Class 16 - Crimson
            [0, 128, 128],      # Class 17 - Teal
            [128, 128, 0],      # Class 18 - Olive
            [128, 0, 128],      # Class 19 - Purple
        ]

# ===============================================================================
# 🎬 VIDEO PROCESSING WITH PERFORMANCE OPTIMIZATION
# ===============================================================================
    def process_video_with_performance_optimization(self, video_path, output_path=None):
        """
        🎬 VIDEO PROCESSING WITH COMPLETE PERFORMANCE OPTIMIZATION

        🔧 PERFORMANCE CRITICAL FEATURES:
        - Frame batching for efficiency
        - Memory management for long videos
        - Progress tracking and ETA calculation
        - Optimized video encoding
        - Real-time performance monitoring
        """
        print(f"\n🎬 PROCESSING VIDEO WITH PERFORMANCE OPTIMIZATION")
        print("=" * 70)

        if not self.pretrained_model:
            print("❌ No model loaded!")
            return None

        # Video setup
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            return None

        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"📹 Video properties:")
        print(f"   📏 Resolution: {width}×{height}")
        print(f"   🎬 FPS: {fps}")
        print(f"   📊 Total frames: {total_frames}")
        print(f"   ⏱️  Duration: {total_frames/fps:.1f} seconds")

        # Setup output
        if output_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = os.path.join(self.dirs['outputs'], f"segmented_{timestamp}.mp4")

        # 🔧 PERFORMANCE OPTIMIZATION: Video encoding settings
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Efficient codec
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        print(f"💾 Output: {output_path}")

        # 🔧 PERFORMANCE OPTIMIZATION: Processing settings
        batch_size = self.processing_config['batch_size']

        print(f"\n🔧 PERFORMANCE SETTINGS:")
        print(f"   📦 Batch size: {batch_size}")
        print(f"   📏 Processing size: {self.processing_config['input_size']}")
        print(f"   💻 Device: {self.device}")

        # Process video with performance monitoring
        frame_count = 0
        start_time = time.time()
        fps_history = []

        with tqdm(total=total_frames, desc="🎬 Processing", unit="frames") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame_start = time.time()

                # Process frame
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Run segmentation
                result = self._process_single_frame_optimized(frame_rgb)
                if result is None:
                    continue

                # Convert back and write
                output_frame = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
                out.write(output_frame)

                # Performance tracking
                frame_time = time.time() - frame_start
                frame_fps = 1.0 / frame_time if frame_time > 0 else 0
                fps_history.append(frame_fps)

                # Keep only recent FPS measurements
                if len(fps_history) > 30:
                    fps_history.pop(0)

                frame_count += 1

                # Update progress with performance info
                if frame_count % 10 == 0:
                    elapsed = time.time() - start_time
                    avg_fps = np.mean(fps_history)
                    remaining_frames = total_frames - frame_count
                    eta = remaining_frames / avg_fps if avg_fps > 0 else 0

                    pbar.set_postfix({
                        'FPS': f'{avg_fps:.1f}',
                        'ETA': f'{eta/60:.1f}m'
                    })

                pbar.update(1)

        # Cleanup and summary
        cap.release()
        out.release()

        total_time = time.time() - start_time
        avg_fps = frame_count / total_time
        file_size = os.path.getsize(output_path) / 1024 / 1024

        print(f"\n🎉 VIDEO PROCESSING COMPLETED!")
        print("=" * 50)
        print(f"⏱️  Total time: {total_time/60:.1f} minutes")
        print(f"📊 Average FPS: {avg_fps:.1f}")
        print(f"📁 Output file: {output_path}")
        print(f"💾 File size: {file_size:.1f} MB")
        print(f"🚀 Throughput: {(width*height*frame_count)/(total_time*1000000):.1f} MP/s")

        return output_path

    def _process_single_frame_optimized(self, frame_rgb):
        """Process single frame with optimization"""
        try:
            # Convert to PIL
            image_pil = Image.fromarray(frame_rgb)

            # Preprocess
            inputs = self.processor(image_pil, return_tensors="pt")
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Inference with optimization
            with torch.no_grad():
                if torch.cuda.is_available():
                    with torch.autocast(device_type='cuda', dtype=torch.float16):
                        outputs = self.pretrained_model(**inputs)
                else:
                    outputs = self.pretrained_model(**inputs)

            # Post-process
            predictions = F.interpolate(
                outputs.logits,
                size=image_pil.size[::-1],
                mode=self.processing_config['interpolation_mode'],
                align_corners=self.processing_config['align_corners']
            )

            predicted_map = predictions.squeeze().cpu().numpy().argmax(axis=0)

            # Create visualization
            colored_mask = self._create_quick_visualization(predicted_map)

            # Blend
            result = cv2.addWeighted(
                frame_rgb,
                self.processing_config['blend_alpha'],
                colored_mask,
                1 - self.processing_config['blend_alpha'],
                0
            )

            return result

        except Exception as e:
            print(f"⚠️ Frame processing error: {e}")
            return None

    def _create_quick_visualization(self, prediction_map):
        """Create quick visualization for video processing"""
        h, w = prediction_map.shape
        colored_mask = np.zeros((h, w, 3), dtype=np.uint8)

        for class_id in np.unique(prediction_map):
            if class_id < len(self.performance_colors):
                mask = prediction_map == class_id
                colored_mask[mask] = self.performance_colors[class_id]

        return colored_mask

# ===============================================================================
# 🖥️ DISPLAY AND SAVING FUNCTIONS
# ===============================================================================
    def _display_complete_results(self, original, visualizations):
        """Display comprehensive results"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Complete Segmentation Results - Performance Optimized', fontsize=16, fontweight='bold')

        axes[0, 0].imshow(original)
        axes[0, 0].set_title('Original Image', fontweight='bold')
        axes[0, 0].axis('off')

        axes[0, 1].imshow(visualizations['colored_mask'])
        axes[0, 1].set_title('Class Segmentation', fontweight='bold')
        axes[0, 1].axis('off')

        axes[0, 2].imshow(visualizations['enhanced_mask'])
        axes[0, 2].set_title('Enhanced with Boundaries', fontweight='bold')
        axes[0, 2].axis('off')

        axes[1, 0].imshow(visualizations['blend_standard'])
        axes[1, 0].set_title('Standard Overlay', fontweight='bold')
        axes[1, 0].axis('off')

        axes[1, 1].imshow(visualizations['blend_enhanced'])
        axes[1, 1].set_title('Enhanced Overlay', fontweight='bold')
        axes[1, 1].axis('off')

        axes[1, 2].imshow(visualizations['confidence_overlay'])
        axes[1, 2].set_title('Confidence Map', fontweight='bold')
        axes[1, 2].axis('off')

        plt.tight_layout()
        plt.show()

    def _save_complete_results(self, image_path, original, visualizations, class_stats, processing_time):
        """Save complete results with performance analysis"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_dir = os.path.join(self.dirs['results'], f"segmentation_{timestamp}")
        os.makedirs(save_dir, exist_ok=True)

        # Save visualizations
        cv2.imwrite(os.path.join(save_dir, "01_original.jpg"),
                   cv2.cvtColor(original, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(save_dir, "02_segmentation.jpg"),
                   cv2.cvtColor(visualizations['colored_mask'], cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(save_dir, "03_enhanced.jpg"),
                   cv2.cvtColor(visualizations['enhanced_mask'], cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(save_dir, "04_overlay.jpg"),
                   cv2.cvtColor(visualizations['blend_enhanced'], cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(save_dir, "05_confidence.jpg"),
                   cv2.cvtColor(visualizations['confidence_overlay'], cv2.COLOR_RGB2BGR))

        # Save performance report
        report = {
            'image_path': image_path,
            'processing_time': processing_time,
            'model_info': self.model_info,
            'processing_config': self.processing_config,
            'class_statistics': class_stats,
            'timestamp': timestamp
        }

        with open(os.path.join(save_dir, "performance_report.json"), 'w') as f:
            json.dump(report, f, indent=2)

        return save_dir

# ===============================================================================
# 🚀 MAIN INTERFACE WITH PERFORMANCE OPTIMIZATION
# ===============================================================================
def main_interface_with_performance():
    """
    🚀 MAIN INTERFACE WITH COMPLETE PERFORMANCE OPTIMIZATION

    This interface provides access to all segmentation features with
    detailed explanations and performance optimization options.
    """
    print("🔥 COMPLETE SEGMENTATION SYSTEM - PERFORMANCE OPTIMIZED")
    print("=" * 80)
    print("✅ Every process step explained in detail")
    print("✅ Performance optimization annotations")
    print("✅ Real-time performance monitoring")
    print("✅ Professional quality results")

    # Initialize system
    system = CompleteSegmentationExplained()

    while True:
        try:
            print(f"\n🎯 PERFORMANCE-OPTIMIZED SEGMENTATION MENU")
            print("=" * 50)
            print("1. 📤 Load model with performance optimization")
            print("2. 🖼️  Process single image (complete explanation)")
            print("3. 🎬 Process video (performance optimized)")
            print("4. ⚙️  Configure performance settings")
            print("5. 📊 Performance benchmark test")
            print("6. 💡 Show performance optimization tips")
            print("0. 🚪 Exit")

            choice = input("\n👉 Choose option (0-6): ").strip()

            if choice == '0':
                print("👋 Segmentation system closed!")
                break

            elif choice == '1':
                print("\n📤 LOADING MODEL WITH PERFORMANCE OPTIMIZATION")
                success = system.load_model_with_performance_optimization()
                if success:
                    print("✅ Model loaded with performance optimizations!")

            elif choice == '2':
                if not system.pretrained_model:
                    print("❌ Load a model first (option 1)")
                    continue

                print("\n🖼️ SINGLE IMAGE PROCESSING - COMPLETE EXPLANATION")
                image_path = input("📁 Enter image path: ").strip()
                if not os.path.exists(image_path):
                    print("❌ Image file not found!")
                    continue

                save_results = input("💾 Save results? (y/n): ").strip().lower() == 'y'

                result = system.process_image_complete_explanation(image_path, save_results)

                if result:
                    print(f"\n🎉 PROCESSING COMPLETED!")
                    print(f"⏱️  Time: {result['performance']['total_time']:.2f}s")
                    print(f"🚀 Throughput: {result['performance']['throughput_mpps']:.1f} MP/s")

            elif choice == '3':
                if not system.pretrained_model:
                    print("❌ Load a model first (option 1)")
                    continue

                print("\n🎬 VIDEO PROCESSING - PERFORMANCE OPTIMIZED")
                video_path = input("📹 Enter video path: ").strip()
                if not os.path.exists(video_path):
                    print("❌ Video file not found!")
                    continue

                output_path = input("💾 Output path (Enter for auto): ").strip()
                if not output_path:
                    output_path = None

                result = system.process_video_with_performance_optimization(video_path, output_path)

                if result:
                    print(f"\n🎉 VIDEO PROCESSING COMPLETED!")
                    print(f"📁 Output: {result}")

            elif choice == '4':
                print("\n⚙️ PERFORMANCE CONFIGURATION")
                print("Current settings:")
                for key, value in system.processing_config.items():
                    print(f"  {key}: {value}")

                print("\n🔧 CRITICAL PERFORMANCE PARAMETERS:")
                print("  • input_size: Higher = better quality, slower")
                print("  • confidence_threshold: Higher = cleaner results")
                print("  • interpolation_mode: bilinear vs nearest")

                # Allow configuration changes
                new_size = input(f"New input size ({system.processing_config['input_size']}): ").strip()
                if new_size:
                    system.processing_config['input_size'] = int(new_size)

                new_threshold = input(f"Confidence threshold ({system.processing_config['confidence_threshold']}): ").strip()
                if new_threshold:
                    system.processing_config['confidence_threshold'] = float(new_threshold)

                print("✅ Configuration updated!")

            elif choice == '5':
                if not system.pretrained_model:
                    print("❌ Load a model first (option 1)")
                    continue

                print("\n📊 PERFORMANCE BENCHMARK TEST")
                print("This will test processing speed on different image sizes")

                test_sizes = [256, 512, 1024]

                for size in test_sizes:
                    print(f"\n🧪 Testing {size}×{size} images...")

                    # Create test image
                    test_image = np.random.randint(0, 255, (size, size, 3), dtype=np.uint8)
                    test_pil = Image.fromarray(test_image)

                    # Time processing
                    start_time = time.time()
                    inputs = system.processor(test_pil, return_tensors="pt")
                    inputs = {k: v.to(system.device) for k, v in inputs.items()}

                    with torch.no_grad():
                        outputs = system.pretrained_model(**inputs)

                    processing_time = time.time() - start_time
                    throughput = (size * size) / processing_time / 1000000

                    print(f"   ⏱️  Time: {processing_time:.3f}s")
                    print(f"   🚀 Throughput: {throughput:.1f} MP/s")

            elif choice == '6':
                print("\n💡 PERFORMANCE OPTIMIZATION TIPS")
                print("=" * 50)
                print("🚀 SPEED OPTIMIZATION:")
                print("  • Use smaller input sizes (256-512)")
                print("  • Enable GPU processing")
                print("  • Use model compilation (PyTorch 2.0+)")
                print("  • Enable mixed precision (fp16)")
                print("  • Use smaller model variants (b0, b1)")

                print("\n🎯 QUALITY OPTIMIZATION:")
                print("  • Use larger input sizes (1024+)")
                print("  • Higher confidence thresholds")
                print("  • Better interpolation (bilinear)")
                print("  • Larger model variants (b3, b4, b5)")

                print("\n💾 MEMORY OPTIMIZATION:")
                print("  • Reduce batch size")
                print("  • Use gradient checkpointing")
                print("  • Clear cache between operations")
                print("  • Process videos in chunks")

            else:
                print("❌ Invalid choice!")

        except KeyboardInterrupt:
            print("\n⏹️ Interrupted by user")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# ===============================================================================
# 🏃 EXECUTION
# ===============================================================================
if __name__ == "__main__":
    main_interface_with_performance()

# ===============================================================================
# 📚 COMPLETE USAGE GUIDE
# ===============================================================================
"""
🔥 COMPLETE SEGMENTATION SYSTEM - USAGE GUIDE

🎯 WHAT THIS SYSTEM DOES:
✅ Loads your pre-trained segmentation model
✅ Explains every processing step in detail
✅ Optimizes performance for speed and quality
✅ Processes images and videos with professional results
✅ Provides detailed performance analysis
✅ Creates multiple visualization types

🚀 PERFORMANCE CRITICAL PARAMETERS:

1. INPUT SIZE (Most Important)
   • 256×256: Very fast, lower quality
   • 512×512: Balanced speed/quality (RECOMMENDED)
   • 1024×1024: High quality, slower
   • 2048×2048: Best quality, very slow

2. MODEL ARCHITECTURE
   • segformer-b0: Fastest (4x speed)
   • segformer-b1: Balanced (3x speed)
   • segformer-b2: Good quality (2x speed)
   • segformer-b3: Better quality (1.5x speed)
   • segformer-b4: Excellent (1x speed)
   • segformer-b5: Best quality (0.7x speed)

3. DEVICE OPTIMIZATION
   • GPU: 10-100x faster than CPU
   • Mixed precision (fp16): 2x speed boost
   • Model compilation: 20-30% speed boost

4. POST-PROCESSING
   • Confidence threshold: Filter weak predictions
   • Interpolation mode: bilinear vs nearest
   • Boundary thickness: Visual clarity

🎬 VIDEO PROCESSING TIPS:
• Use input_size=512 for good balance
• Enable GPU for real-time processing
• Monitor memory usage for long videos
• Use batch processing for efficiency

📊 EXPECTED PERFORMANCE:
• GPU (RTX 3080): ~50-100 FPS at 512×512
• GPU (RTX 4090): ~100-200 FPS at 512×512
• CPU (Modern): ~2-5 FPS at 512×512

🎯 QUALITY IMPROVEMENTS:
• Higher input resolution
• Better model architecture
• Confidence filtering
• Boundary enhancement
• Multi-scale processing
"""